## 2D subduction


This code was written by Dan Sandiford, Louis Moresi and the Underworld Team. It is licensed under the Creative Commons Attribution 4.0 International License . We offer this licence to encourage you to modify and share the examples and use them to help you in your research.

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />


In [1]:
#If run through Docker we'll point at the local 'unsupported dir.'
#On hpc, the path should also include a directory holding the unsupported_dan.
import os
import sys

if os.getcwd() == '/workspace/newSlab':
    sys.path.append('../unsupported')

#this does't actually need to be protected. More a reminder it's an interim measure
try:
    sys.path.append('../unsupported')
except:
    pass
    

In [2]:
import numpy as np
import underworld as uw
import math
from underworld import function as fn
import glucifer
import os
import sys
from easydict import EasyDict as edict
import operator
import pickle


#
from unsupported_dan.utilities.interpolation import nn_evaluation
from unsupported_dan.interfaces.marker2D import markerLine2D
from unsupported_dan.faults.faults2D import fault2D, fault_collection
from unsupported_dan.alchemy.materialGraph import MatGraph
from unsupported_dan.easymodels.checkpoint import checkpoint
from unsupported_dan.utilities.misc import cosine_taper
from unsupported_dan.utilities.subduction import slab_top
from unsupported_dan.utilities.mechanics import eig2d
from unsupported_dan.utilities.phase_utils import phases
from unsupported_dan.interfaces.smoothing2D import *







../unsupported/unsupported_dan/utilities/__init__.py:9: UserWarning: 

The utilities module is not supported.
Questions should be addressed to sandd@student.unimelb.edu.au 
 
  Questions should be addressed to sandd@student.unimelb.edu.au \n """
../unsupported/unsupported_dan/interfaces/__init__.py:9: UserWarning: 

The interface module is not supported.
Questions should be addressed to louis.moresi@unimelb.edu.au 
 
  Questions should be addressed to louis.moresi@unimelb.edu.au \n """
../unsupported/unsupported_dan/faults/__init__.py:9: UserWarning: 

The fault module is not supported.
Questions should be addressed to louis.moresi@unimelb.edu.au 
 
  Questions should be addressed to louis.moresi@unimelb.edu.au \n """
../unsupported/unsupported_dan/alchemy/__init__.py:9: UserWarning: 

The alchemy module is not supported.
Questions should be addressed to sandd@student.unimelb.edu.au 
 
  Questions should be addressed to sandd@student.unimelb.edu.au \n """
../unsupported/unsupported_dan

In [3]:
#import ipython
#IPython.__version__

## Setup output directories

In [4]:
############
#Model letter and number
############


#Model letter identifier demarker
Model = "T"

#Model number identifier demarker:
ModNum = 2

#Any isolated letter / integer command line args are interpreted as Model/ModelNum

if len(sys.argv) == 1:
    ModNum = ModNum 
elif sys.argv[1] == '-f': #
    ModNum = ModNum 
else:
    for farg in sys.argv[1:]:
        if not '=' in farg: #then Assume it's a not a paramter argument
            try:
                ModNum = int(farg) #try to convert everingthing to a float, else remains string
            except ValueError:
                Model  = farg

In [5]:
###########
#Standard output directory setup
###########

outputPath = "results" + "/" +  str(Model) + "/" + str(ModNum) + "/" 
imagePath = outputPath + 'images/'
filePath = outputPath + 'files/'
#checkpointPath = outputPath + 'checkpoint/'
dbPath = outputPath + 'gldbs/'
xdmfPath = outputPath + 'xdmf/'
outputFile = 'results_model' + Model + '_' + str(ModNum) + '.dat'

if uw.rank()==0:
    # make directories if they don't exist
    if not os.path.isdir(outputPath):
        os.makedirs(outputPath)
    if not os.path.isdir(imagePath):
        os.makedirs(imagePath)
    if not os.path.isdir(dbPath):
        os.makedirs(dbPath)
    if not os.path.isdir(filePath):
        os.makedirs(filePath)
    if not os.path.isdir(xdmfPath):
        os.makedirs(xdmfPath)
        
uw.barrier() #Barrier here so no procs run the check in the next cell too early

In [6]:
#*************CHECKPOINT-BLOCK**************#

#cp = checkpoint(outputPath + 'checkpoint/', loadpath='./results/A/1/checkpoint/10')
cp = checkpoint(outputPath + 'checkpoint/')


#*************CHECKPOINT-BLOCK**************#


In [7]:
#cp.dictDict, cp.objDict
#cp.dictDict

## Model parameters and scaling

In [8]:
#dp.potentialTemp - 273

In [9]:
dp = edict({})
#Main physical paramters
dp.depth=1000e3                         #Depth
dp.refDensity=3300.                        #reference density
dp.refGravity=9.8                          #surface gravity
dp.viscosityScale=1e20                       #reference upper mantle visc., 
dp.refDiffusivity=1e-6                     #thermal diffusivity
dp.refExpansivity=3e-5                     #surface thermal expansivity
dp.gasConstant=8.314                    #gas constant
dp.specificHeat=1250.                   #Specific heat (Jkg-1K-1)
dp.potentialTemp=1573.                  #mantle potential temp (K)
dp.surfaceTemp=273.                     #surface temp (K)
#Rheology - flow law paramters
dp.cohesionMantle=20e6                   #mantle cohesion in Byerlee law
dp.cohesionInterface=1e6                    #crust cohesion in Byerlee law
dp.frictionMantle=0.2                   #mantle friction coefficient in Byerlee law (tan(phi))
dp.frictionInterface=0.02                   #crust friction coefficient 
dp.diffusionPreExp=5.34e-10             #1./1.87e9, pre-exp factor for diffusion creep
dp.diffusionEnergy=3e5 
dp.diffusionVolume=5e-6
dp.lowerMantlePreExp=4.23e-15           #1./2.36e14
dp.lowerMantleEnergy=2.0e5
dp.lowerMantleVolume=1.5e-6
dp.lowerMantleViscFac = 30.
#Fk approach to interface viscosity
dp.delViscInterface = 1e4
dp.refViscInterface = 1e20
dp.refDepthInterface = 80e3
#power law creep params
dp.powerLawStrain = 1e-15
dp.powerLawExp = 3.5

#Rheology - cutoff values
dp.viscosityMin=1e18
dp.viscosityMax=1e25                #viscosity max in the mantle material
dp.viscosityMinInterface=1e18               #viscosity min in the weak-crust / interface material
dp.viscosityMaxInterface=1e25               #viscosity min in the weak-crust / interface material
dp.ysMaxInterface = 50e6
dp.yieldStressMax=300*1e6              #
dp.interfaceViscCutoffDepth = 100e3
dp.interfaceViscEndWidth = 20e3     

#Intrinsic Lengths
dp.mantleCrustDepth=10.*1e3              #Crust depth
dp.faultThickness = 10.*1e3              #interface material (crust) an top of slabs
dp.lowerMantleDepth=660.*1e3  
dp.interfaceDestroyDepth=400.*1e3             #Beyond this depth, crust, interface, etc get destroyed
dp.interfaceResetDepth=400.*1e3               #Beyond this depth there is no proximity accumulation()

#Slab and plate init. parameters
dp.subZoneLoc=-100e3                    #X position of subduction zone...km
dp.maxDepth=150e3
dp.radiusOfCurv = 350e3                          #radius of curvature
dp.slabMaxAge=70e6                     #age of subduction plate at trench
dp.opMaxAge=35e6                       #age of op
dp.spAgeGrad = (-1./4)  # (y / cm.) The gradient direction is taken as away from the sz
dp.opAgeGrad = (-1./8)  # (y / cm.) The gradient direction is taken as away from the sz
#Misc
dp.stickyAirDepth=100e3                 #depth of sticky air layer
dp.viscosityStickyAir=1e19              #stick air viscosity, normal
#derived params
dp.deltaTemp = dp.potentialTemp-dp.surfaceTemp
dp.tempGradMantle = (dp.refExpansivity*dp.refGravity*(dp.potentialTemp))/dp.specificHeat
dp.tempGradSlab = (dp.refExpansivity*dp.refGravity*(dp.surfaceTemp + 400.))/dp.specificHeat

#a velocity scale, used in some caes to set an approx viscosity in the interface
dp.subVelocity = 4*(1/100.)*(1./(3600*24*365)) #m/s

#op velocity will be set as a BC, or False
dp.opVelocity = -1.*(1/100.)*(1./(3600*24*365)) 


#Modelling and Physics switches

md = edict({})
md.refineMeshStatic=True
md.stickyAir=False
md.aspectRatio=5.
md.res=48
md.ppc=35                                 #particles per cell
md.elementType="Q1/dQ0"
#md.elementType="Q2/DPC1"
md.secInvFac=math.sqrt(1.)
md.courantFac=0.5                         #extra limitation on timestepping
md.thermal = True                        #thermal system or compositional
md.swarmInitialFac = 0.6                 #initial swarm layout will be int(md.ppc*md.swarmInitialFac), popControl will densify later
md.compBuoyancy = False
md.nltol = 0.01
md.maxSteps = 20000
md.checkpointEvery = 200
md.swarmUpdate = 10
md.druckerAlpha = 0.
md.druckerAlphaFault = 0.
md.penaltyMethod = True
md.opuniform = False
md.spuniform = False
md.opfixed = False
md.spfixed = False
md.buoyancyFac = 1.0
md.restartParams = True #whether we load params on checkpoint restart. md.restartParams = True #whether we load params on checkpoint restart. 
#The following are time-based actions
md.filesMy = 1.0e6 #dimensional time interval to write files
md.diffuseInitial =  0. #5e6 # years to run initial diffusion for. Or set to zero.

#some flags mostlt controlling the interface viscosity. 
md.interfaceType = 2 #1 for 'weak' crust, 2 for 'fault ' (isotropic)
md.plasticInterface = 1 #non linear (plastic) of linear (effective viscosity)
md.faultHeal=1
md.faultLocFac = 1. #this is the relative location of the fault in terms of the fault thickess from the top of slab
md.powerLaw = False
md.viscCombine = 1 #zero here will use min(), anythin else toggles harmonic
md.fixedRidges = False
md.phaseBuoyancy = False


In [10]:
#1/40.

In [11]:
#dp.maxDepth=110e3
#dp.radiusOfCurv = 35e3
dp.tempGradMantle

0.0003699696000000001

####TEST BLOCK, smaller activation energy

fracE = 0.6 #We want to multiply the activation Energy by this value
delE= dp.diffusionEnergy - dp.diffusionEnergy*fracE
dp.diffusionEnergy *= fracE
dp.diffusionVolume *=0.8
dp.diffusionPreExp /= np.exp(delE /(dp.gasConstant*dp.potentialTemp))

In [12]:
uw.barrier()

In [13]:
##Parse any command-line args

from unsupported_dan.easymodels import easy_args
sysArgs = sys.argv

#We want to run this on both the paramter dict, and the model dict
easy_args(sysArgs, dp)
easy_args(sysArgs, md)

In [14]:
uw.barrier()


In [15]:
sf = edict({})

sf.lengthScale=2900e3
sf.viscosityScale = dp.viscosityScale
sf.stress = (dp.refDiffusivity*sf.viscosityScale)/sf.lengthScale**2
#sf.lithGrad = dp.refDensity*dp.refGravity*(sf.lengthScale)**3/(sf.viscosityScale*dp.refDiffusivity) 
sf.lithGrad = (sf.viscosityScale*dp.refDiffusivity) /(dp.refDensity*dp.refGravity*(sf.lengthScale)**3)
sf.velocity = dp.refDiffusivity/sf.lengthScale
sf.strainRate = dp.refDiffusivity/(sf.lengthScale**2)
sf.time = 1./sf.strainRate
sf.actVolume = (dp.gasConstant*dp.deltaTemp)/(dp.refDensity*dp.refGravity*sf.lengthScale)
sf.actEnergy = (dp.gasConstant*dp.deltaTemp)
sf.diffusionPreExp = 1./sf.viscosityScale
sf.deltaTemp  = dp.deltaTemp
sf.pressureDepthGrad = (dp.refDensity*dp.refGravity*sf.lengthScale**3)/(dp.viscosityScale*dp.refDiffusivity)


#dimesionless params
ndp  = edict({})

ndp.rayleigh = md.buoyancyFac*(dp.refExpansivity*dp.refDensity*dp.refGravity*dp.deltaTemp*sf.lengthScale**3)/(dp.viscosityScale*dp.refDiffusivity)

#Take care with these definitions, 
ndp.surfaceTemp = dp.surfaceTemp/sf.deltaTemp  #Ts
ndp.potentialTemp = dp.potentialTemp/sf.deltaTemp - ndp.surfaceTemp #Tp' = Tp - TS
ndp.tempGradMantle = dp.tempGradMantle/(sf.deltaTemp/sf.lengthScale)
ndp.tempGradSlab = dp.tempGradSlab/(sf.deltaTemp/sf.lengthScale)

#lengths / distances
ndp.depth = dp.depth/sf.lengthScale
ndp.leftLim = -0.5*ndp.depth*md.aspectRatio
ndp.rightLim = 0.5*ndp.depth*md.aspectRatio
ndp.faultThickness = dp.faultThickness/sf.lengthScale
ndp.mantleCrustDepth =  dp.mantleCrustDepth/sf.lengthScale
ndp.interfaceDestroyDepth = dp.interfaceDestroyDepth/sf.lengthScale
ndp.interfaceResetDepth = dp.interfaceResetDepth/sf.lengthScale
ndp.lowerMantleDepth = dp.lowerMantleDepth/sf.lengthScale


#times - for convenience and sanity the dimensional values are in years, conversion to seconds happens here
ndp.slabMaxAge =  dp.slabMaxAge*(3600*24*365)/sf.time
ndp.opMaxAge = dp.opMaxAge*(3600*24*365)/sf.time

#Rheology - flow law paramters
ndp.cohesionMantle=dp.cohesionMantle/sf.stress                  #mantle cohesion in Byerlee law
ndp.cohesionInterface=dp.cohesionInterface/sf.stress                  #crust cohesion in Byerlee law
ndp.frictionMantle=dp.frictionMantle/sf.lithGrad                  #mantle friction coefficient in Byerlee law (tan(phi))
ndp.frictionInterface=dp.frictionInterface/sf.lithGrad                  #crust friction coefficient 
ndp.diffusionPreExp=dp.diffusionPreExp/sf.diffusionPreExp                #pre-exp factor for diffusion creep
ndp.diffusionEnergy=dp.diffusionEnergy/sf.actEnergy
ndp.diffusionVolume=dp.diffusionVolume/sf.actVolume
ndp.lowerMantlePreExp=dp.lowerMantlePreExp/sf.diffusionPreExp 
ndp.lowerMantleEnergy=dp.lowerMantleEnergy/sf.actEnergy
ndp.lowerMantleVolume=dp.lowerMantleVolume/sf.actVolume
ndp.yieldStressMax=dp.yieldStressMax/sf.stress 
#Fk approach to interface viscosity
ndp.logDelVisc = np.log(dp.delViscInterface)
ndp.refViscInterface = dp.refViscInterface/sf.viscosityScale
ndp.refDepthInterface = dp.refDepthInterface/sf.lengthScale
#
ndp.powerLawStrain = dp.powerLawStrain/sf.strainRate
ndp.powerLawExp = dp.powerLawExp

#Rheology - cutoff values
ndp.viscosityMin= dp.viscosityMin /sf.viscosityScale
ndp.viscosityMax=dp.viscosityMax/sf.viscosityScale
ndp.viscosityMinInterface= dp.viscosityMinInterface /sf.viscosityScale
ndp.viscosityMaxInterface= dp.viscosityMaxInterface /sf.viscosityScale
ndp.lowerMantleViscFac = dp.lowerMantleViscFac
ndp.interfaceViscCutoffDepth = dp.interfaceViscCutoffDepth/sf.lengthScale
ndp.interfaceViscEndWidth = dp.interfaceViscEndWidth/sf.lengthScale
ndp.ysMaxInterface  = dp.ysMaxInterface/sf.stress


#Slab and plate init. parameters
ndp.subZoneLoc = dp.subZoneLoc/sf.lengthScale
ndp.maxDepth = dp.maxDepth/sf.lengthScale
ndp.radiusOfCurv = dp.radiusOfCurv/sf.lengthScale
ndp.subVelocity = dp.subVelocity/sf.velocity
ndp.opVelocity = dp.opVelocity/sf.velocity
ndp.spAgeGrad = dp.spAgeGrad* (3600*24*365)*(100.)*(sf.lengthScale/sf.time)
ndp.opAgeGrad = dp.opAgeGrad* (3600*24*365)*(100.)*(sf.lengthScale/sf.time)



In [16]:
#1. - (70/2900.)

In [17]:
#dp.subVelocity

#ndp.tempGradMantle

In [18]:
#sf.time
#ndp.opAgeGrad
#sf.time/(3600*24*365*1e6)
#15.9/(ndp.diffusionEnergy)
#ndp.tempGradMantle

1. - (100./2900)

0.9655172413793104

In [19]:
#*************CHECKPOINT-BLOCK**************#


#if restart, attempt to read in saved dicts. 
if cp.restart and md.restartParams:
    try:
        with open(os.path.join(cp.loadpath, 'dp.pkl'), 'rb') as fp:
                            dp = pickle.load(fp)
        with open(os.path.join(cp.loadpath, 'sf.pkl'), 'rb') as fp:
                            sf = pickle.load(fp)
        with open(os.path.join(cp.loadpath, 'md.pkl'), 'rb') as fp:
                            md = pickle.load(fp)

    except:
        print("couldn't load paramter dictionaries on restart")


    
#add dicts to the checkpointinng object
cp.addDict(dp, 'dp')
cp.addDict(sf, 'sf')
cp.addDict(md, 'md')
#*************CHECKPOINT-BLOCK**************#


In [20]:
#1. - (100e3/sf.lengthScale)

## Inform checkpoint object of values we'll be tracking


In [21]:
#Metrics / values we want to track and save get stored in a dictionary of lists.
#By adding this dictionary to the checkpoint object, 
#it becomes part of regular checkpointing proceedure

#It is likely we'll want to update and save the dictionary more often that checkpointing frequency
#in this case, we could pickle the dictionary, or use np.savez(fname, **valuesDict)

#to update a value-list in the dictionary, just use:
#valuesDict.valueList2.append(newValue)

#*************CHECKPOINT-BLOCK**************#

if not cp.restart:
    valuesDict = edict({})
    valuesDict.timeAtSave = []
    valuesDict.stepAtSave = []
    valuesDict.maxVxSurf = []
    valuesDict.minVxSurf = []
    valuesDict.areaintLith = []
    valuesDict.vdintLith = []
    valuesDict.vdintUM = []
    valuesDict.vdintLM = []
    valuesDict.areaintSlabLM = []
    valuesDict.vdintSlabLM = []
    valuesDict.areaintInterface = []
    valuesDict.vdintInterface= []
    valuesDict.potentialWork= []
    valuesDict.vdintLower= []
    valuesDict.areaintLower= []
    valuesDict.vdintLowerBending= []
    valuesDict.areaintLowerBending= []
    valuesDict.subZoneLoc = []
    valuesDict.spRidgeLoc = []
    valuesDict.opRidgeLoc = []
    valuesDict.wedgeHotCornerX = []
    valuesDict.wedgeHotCornerY = []
    valuesDict.tempInt = []
    valuesDict.nusseltTop = []
    valuesDict.critWedgeTempArea = []


if cp.restart:
    try:
        with open(os.path.join(cp.loadpath, 'vd.pkl'), 'rb') as fp:
            valuesDict= pickle.load(fp)
    except:
        print("couldn't load paramter dictionaries on restart")
        

cp.addDict(valuesDict, 'vd')
#*************CHECKPOINT-BLOCK**************#

In [22]:
#define the location of plate boundaries


#if not cp.restart:
subZoneLoc = fn.misc.constant(ndp.subZoneLoc)
spRidgeLoc = fn.misc.constant(ndp.leftLim)
opRidgeLoc = fn.misc.constant(ndp.rightLim)

if cp.restart:
    try:
        subZoneLoc = fn.misc.constant(valuesDict.subZoneLoc[-1]) #this one carries the upadatin' values
        spRidgeLoc = fn.misc.constant(valuesDict.spRidgeLoc[-1])
        opRidgeLoc = fn.misc.constant(valuesDict.opRidgeLoc[-1])
    except:
        pass


## Build Mesh

In [23]:
#Domain and Mesh paramters
yres = int(md.res)
xres = int(md.res*12) 



mesh = uw.mesh.FeMesh_Cartesian( elementType = (md.elementType),
                                 elementRes  = (xres, yres), 
                                 minCoord    = (ndp.leftLim, 1. - ndp.depth), 
                                 maxCoord    = (ndp.rightLim, 1.)) 

velocityField   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
pressureField   = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
temperatureField    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )

if md.thermal:
    temperatureDotField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 ) #create this only if Adv-diff
    diffusivityFn = fn.misc.constant(1.)
    
    
    
# Any extra mesh vars. we want to define (mostly to facilite saving as xdmf)
strainRateField    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
viscosityField    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
    


In [52]:
#*************CHECKPOINT-BLOCK**************#
cp.addObject(velocityField,'velocityField')
cp.addObject(pressureField,'pressureField')
if md.thermal:
    cp.addObject(temperatureField,'temperatureField')
    cp.addObject(temperatureDotField,'temperatureDotField')
    

#*************CHECKPOINT-BLOCK**************#

In [53]:
#print(cp.objDict.keys())

In [54]:
#*************CHECKPOINT-BLOCK**************#

if cp.restart:
    velocityField.load(cp.loadpath + '/velocityField.h5')
    pressureField.load(cp.loadpath + '/pressureField.h5')
    if md.thermal:
        temperatureField.load(cp.loadpath + '/temperatureField.h5')
        temperatureDotField.load(cp.loadpath + '/temperatureDotField.h5')
#*************CHECKPOINT-BLOCK**************#


## Misc. functions

In [55]:
#miscellaneous Uw functions, geometry measurements

coordinate = fn.input()
depthFn = mesh.maxCoord[1] - coordinate[1] #a function providing the depth


xFn = coordinate[0]  #a function providing the x-coordinate
yFn = coordinate[1]

#We'll use this for integrals

surfaceIndexSet=mesh.specialSets["MaxJ_VertexSet"]
_surfLength  = uw.utils.Integral( 1., mesh=mesh, integrationType='Surface', surfaceIndexSet=surfaceIndexSet)
surfLength = _surfLength.evaluate()[0]


#Create a binary circle
def inCircleFnGenerator(centre, radius):
    coord = fn.input()
    offsetFn = coord - centre
    return fn.math.dot( offsetFn, offsetFn ) < radius**2


###################
#Create integral, max/min templates 
###################

globRFn = fn.misc.constant(1.)

def volumeint(Fn = 1., rFn=globRFn):
    return uw.utils.Integral( Fn*rFn,  mesh )

def surfint(Fn = 1., rFn=globRFn, surfaceIndexSet=mesh.specialSets["MaxJ_VertexSet"]):
    return uw.utils.Integral( Fn*rFn, mesh=mesh, integrationType='Surface', surfaceIndexSet=surfaceIndexSet)




## Static Mesh refinement

In [56]:
if md.refineMeshStatic:
    mesh.reset()

    jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
    yFn = coordinate[1]
    yField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )
    yField.data[:] = 0.
    yBC = uw.conditions.DirichletCondition( variable=yField, indexSetsPerDof=(jWalls,) )

    # set bottom wall temperature bc
    for index in mesh.specialSets["MinJ_VertexSet"]:
        yField.data[index] = mesh.minCoord[1]
    # set top wall temperature bc
    for index in mesh.specialSets["MaxJ_VertexSet"]:
        yField.data[index] = mesh.maxCoord[1]



    s = 4
    intensityFac = 6.
    intensityFn = (((yFn - mesh.minCoord[1])/(mesh.maxCoord[1]-mesh.minCoord[1]))**s)
    intensityFn *= intensityFac
    intensityFn += 1.


    yLaplaceEquation = uw.systems.SteadyStateHeat(temperatureField=yField, fn_diffusivity=intensityFn, conditions=[yBC,])

    # get the demarker heat equation solver
    yLaplaceSolver = uw.systems.Solver(yLaplaceEquation)
    # solve
    yLaplaceSolver.solve()


    #Get the array of Y positions - copy may be necessary, not sure. 
    newYpos = yField.data.copy() 

    uw.barrier()
    with mesh.deform_mesh():
         mesh.data[:,1] = newYpos[:,0]

## Boundary Conditions

In [57]:
#Stokes BCs

iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
tWalls = mesh.specialSets["MaxJ_VertexSet"]
bWalls =mesh.specialSets["MinJ_VertexSet"]
      
        
freeslipBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                               indexSetsPerDof = ( iWalls, jWalls) )

In [58]:
#Set a constat velocity BC on the upper plate
#(having bcs extend right to the side wall was very unstable)

limBCs = ndp.subZoneLoc + (200e3/sf.lengthScale)

checkFn = fn.branching.conditional([(xFn < limBCs, False),
                                       (xFn  > ndp.rightLim - (200e3/sf.lengthScale), False),
                                  (True, True)])


velnodesMask = np.where(checkFn.evaluate(tWalls) == True)[0] #gives the indeces of the tWalls array 
velnodes = tWalls.data[velnodesMask] #gets that actual node nos

velnodeset = mesh.specialSets["Empty"]
velnodeset += velnodes

if ndp.opVelocity:
    freeslipBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                               indexSetsPerDof = ( iWalls + velnodeset, jWalls) )
    
    if velnodeset.data.shape[0]:
        velocityField.data[velnodeset.data, 0] = ndp.opVelocity
    

In [59]:
#Energy BCs

if md.thermal:
    dirichTempBC = uw.conditions.DirichletCondition(     variable=temperatureField, 
                                              indexSetsPerDof=(tWalls,) )
    
    #in this case we imposed a fixed temp along the sidewalls
    if md.fixedRidges:
        dirichTempBC = uw.conditions.DirichletCondition(     variable=temperatureField, 
                                              indexSetsPerDof=(tWalls + iWalls,) )

## Swarm

In [60]:
#Materials
mantleID = 0
crustID = 1
airID = 2      #in case we use sticky air

#list of all material indexes
material_list = [mantleID, crustID, airID]

In [61]:
#*************CHECKPOINT-BLOCK**************#

swarm = uw.swarm.Swarm(mesh=mesh, particleEscape=True)
cp.addObject(swarm,'swarm')
materialVariable      = swarm.add_variable( dataType="int", count=1 )
cp.addObject(materialVariable,'materialVariable')
proxyTempVariable = swarm.add_variable( dataType="double", count=1 )
proximityVariable      = swarm.add_variable( dataType="int", count=1 )
cp.addObject(proximityVariable,'proximityVariable')
ageVariable      = swarm.add_variable( dataType="double", count=1 )
cp.addObject(ageVariable,'ageVariable')

if not md.thermal:
    cp.addObject(proxyTempVariable,'proxyTempVariable')


if cp.restart:
    swarm.load(cp.loadpath + '/swarm.h5')
    materialVariable.load(cp.loadpath + '/materialVariable.h5')
    proximityVariable.load(cp.loadpath + '/proximityVariable.h5')
    ageVariable.load(cp.loadpath + '/ageVariable.h5')
    if not md.thermal:
        proxyTempVariable.load(cp.loadpath + '/proxyTempVariable.h5')   


else:
    layout = uw.swarm.layouts.PerCellRandomLayout(swarm=swarm, particlesPerCell=int(md.ppc*md.swarmInitialFac))
    swarm.populate_using_layout( layout=layout ) # Now use it to populate.
    proxyTempVariable.data[:] = 0.0
    materialVariable.data[:] = mantleID
    proximityVariable.data[:] = 0.0

#*************CHECKPOINT-BLOCK**************#





In [62]:
#These variables don't need checkpointing. They can / should be rebuilt

signedDistanceVariable = swarm.add_variable( dataType="double", count=1 )
#directorVector   = swarm.add_variable( dataType="double", count=2)

#directorVector.data[:,:] = 0.0
signedDistanceVariable.data[:] = 0.0


In [63]:

#Pass this to Figures to see full extent
bBox=((mesh.minCoord[0], mesh.minCoord[1]),(mesh.maxCoord[0], mesh.maxCoord[1]))

## Initial Conditions

In [64]:
#plate depth

#Flags to control plate behaviour
#md.opuniform
#md.spuniform
#md.opfixed
#md.spfixed

#T&S 4.126
thicknessAtTrench = 2.32*math.sqrt(1.*ndp.slabMaxAge)

#the basic plate age Fns, defined by age at temp and age gradient
    
if dp.spAgeGrad:
    spAge = fn.math.abs(ndp.slabMaxAge + ndp.spAgeGrad*fn.math.abs(ndp.subZoneLoc - xFn))
else:
    spAge = ndp.slabMaxAge*fn.math.abs((mesh.minCoord[0] - xFn)/(ndp.subZoneLoc - mesh.minCoord[0]))

if dp.opAgeGrad: #likewise
    opAge = fn.math.abs(ndp.opMaxAge + ndp.opAgeGrad*fn.math.abs(ndp.subZoneLoc - xFn))
else:
    opAge = ndp.opMaxAge*fn.math.abs((mesh.maxCoord[0] - xFn)/(ndp.subZoneLoc - mesh.maxCoord[0]))
 
#
opAge = fn.misc.min(opAge, ndp.opMaxAge)
spAge = fn.misc.min(spAge, ndp.slabMaxAge)



#unless we specify fixing the plate to the sidwalls, we'll inforce a boundary

sig = 150e3/sf.lengthScale
ridgeFn = 1. -  \
                fn.math.exp(-1.*(xFn - mesh.minCoord[0])**2/(2 * sig**2))- \
                fn.math.exp(-1.*(xFn - mesh.maxCoord[0])**2/(2 * sig**2))
if not md.spfixed:
    spAge = ridgeFn*spAge
else:
    pass
        
if not md.opfixed:
    opAge = ridgeFn*opAge
else:
    pass

#we need to apply the sp age in the slab region as well
#this is only needed for ageSp < ageOp. 
#however, this fix only works when circGradientFn is used to define the slab
slabCirc = inCircleFnGenerator((ndp.subZoneLoc, 1.0 - ndp.radiusOfCurv), ndp.radiusOfCurv)

proxyageFn = fn.branching.conditional([(xFn <= ndp.subZoneLoc, spAge), #idea is to make this arbitrarily complex
                                       (slabCirc > 0.9, spAge),
                                  (True, opAge)])

In [65]:
#%pylab inline
#fig, ax = plt.subplots()

#ax.plot(opAge.evaluate(tWalls), c= 'r')
#ax.plot(spAge.evaluate(tWalls), c= 'b')
#ax.hlines(ndp.slabMaxAge, 0, 600 )
#ax.hlines(ndp.opMaxAge, 0, 600 )

## Marker lines  for slab, fault, tracking

In [66]:
#Create some slab gradient functions to use with slab_top()


def linearGradientFn(S):
    return np.tan(np.deg2rad(-45.))


def circGradientFn(S):
    if S == 0.:
        return 0.
    elif S < ndp.radiusOfCurv:
        return max(-S/np.sqrt((ndp.radiusOfCurv**2 - S**2)), -1e3)
    else:
        return -1e5
    
    
def polyGradientFn(S):
    if S == 0.:
        return 0.
    else:
        return -1*(S/ndp.radiusOfCurv)**2
    
    
#def specialFn(S):
#    if S == 0.:
#        return 0.
#    elif S < (20/2900.):
#        return max(-S/np.sqrt((ndp.radiusOfCurv**2 - S**2)), -0.75)
#    else:
#        return -0.75


### slab top

In [67]:
#ndp.ysMaxInterf

In [68]:
ds_fault = 2e3/sf.lengthScale
normal = [1.,0.]

#slabdata = slab_top([ndp.subZoneLoc, 1.0], normal, linearGradientFn, ds_fault, ndp.maxDepth, mesh)
#slabdata= slab_top([ndp.subZoneLoc, 1.0], normal, polyGradientFn, ds_fault, ndp.maxDepth, mesh)
slabdata = slab_top([ndp.subZoneLoc, 1.0], normal, circGradientFn, ds_fault, ndp.maxDepth, mesh)


In [69]:
slabxs = slabdata[:,0]
slabys = slabdata[:,1]

In [70]:
#%pylab inline
#fig, ax = plt.subplots()
#ax.scatter(slabxs, slabys)
#ax.set_aspect('equal')

In [71]:
slabLine = markerLine2D(mesh, velocityField, slabxs, slabys, thicknessAtTrench, 1., insidePt=(-0.5,0.5))

In [72]:
#markerLine2D?

In [73]:
#Assign the signed distance for the slab - in this case we only want the portion where the signed distance is positive

#Note distance=2.*thicknessAtTrench: we actually want to allow distance greater than thicknessAtTrench in the kDTree query, 
#as some of these distances will not be orthogonal to the marker line, the dot product in the function will project these distances onto the normal vector
#We'll cull distances greater than thicknessAtTrench with a numpy boolean slice - this is a big advantage in parallel

sd, pts = slabLine.compute_signed_distance(swarm.particleCoordinates.data, distance=2.*thicknessAtTrench)
signedDistanceVariable.data[np.logical_and(sd>0, sd<=slabLine.thickness)] = sd[np.logical_and(sd>0, sd<=slabLine.thickness)]

In [74]:
slabXConds = operator.and_(xFn > slabxs.min(), xFn < slabxs.max())
slabYConds = depthFn < 1. - slabys.min()

#Two functions we'll use to limit the region of the initial thermal stancil
slabRegion =  fn.branching.conditional([(operator.and_(slabXConds,slabYConds), True),
                          (True, False)])



In [75]:
bufferlength = 1e3/sf.lengthScale

plateDepthFn = fn.branching.conditional([(depthFn < thicknessAtTrench, depthFn),
                                        (True, 1.)])

#plateTempProxFn = fn.math.erf((depthFn*sf.lengthScale)/(2.3*fn.math.sqrt(dp.refDiffusivity*proxyageFn)))
plateTempProxFn = ndp.potentialTemp*fn.math.erf((plateDepthFn)/(2.*fn.math.sqrt(1.*proxyageFn)))

#slabTempProx  = fn.math.erf((signedDistanceVariable*sf.lengthScale)/(2.*fn.math.sqrt(dp.refDiffusivity*proxyageFn)))
slabTempProx  = ndp.potentialTemp*fn.math.erf((signedDistanceVariable)/(2.*np.sqrt(1.*ndp.slabMaxAge)))


proxytempConds = fn.branching.conditional([(signedDistanceVariable < bufferlength, plateTempProxFn),
                          (slabRegion,  fn.misc.min(slabTempProx , plateTempProxFn)),                 

                          (True, plateTempProxFn)]) 


#*************CHECKPOINT-BLOCK**************#

if not cp.restart:
    proxyTempVariable.data[:] = proxytempConds.evaluate(swarm)
    

#*************CHECKPOINT-BLOCK**************#


In [76]:
#cp.restart

### marker

In this notebook, the markerLine helps us set up the weak interface material, as well as tracking various flow metrics

It could also be used it to define a proximity of Transversely Isotropic frictional behaviour

In [77]:
morexs = np.arange(mesh.minCoord[0] + 100e3/sf.lengthScale, ndp.subZoneLoc, ds_fault)[:-1]
moreys = mesh.maxCoord[1]*np.ones(morexs.shape)


In [78]:
#Build marker: copy the slab line, then move using the normal vector (director)
#or, if checkpoint restart, load swarm

markerxs = np.concatenate((morexs,slabxs[:-2]))
markerys = np.concatenate((moreys,slabys[:-2]))


marker = markerLine2D(mesh, velocityField,[], [], ndp.faultThickness,  1, insidePt=(-0.5,0.5))


#*************CHECKPOINT-BLOCK**************#
cp.addObject(marker.swarm,'marker')


if not cp.restart:
    marker.add_points(markerxs, markerys)
    with marker.swarm.deform_swarm():
        marker.swarm.particleCoordinates.data[:] += marker.director.data*ndp.faultThickness
        
else:
    marker.swarm.load(cp.loadpath + '/marker.h5')
#*************CHECKPOINT-BLOCK**************#
    

marker.rebuild()
marker.swarm.update_particle_owners()

In [79]:
#marker.swarm.particleCoordinates.data.shape



In [80]:
#*************CHECKPOINT-BLOCK**************#

In [81]:
print('got to test point')

#uw.barrier()

got to test point


In [82]:
#set the material variable with the markerLine

sd, pts0 = marker.compute_signed_distance(swarm.particleCoordinates.data, distance=ndp.mantleCrustDepth)

#*************CHECKPOINT-BLOCK**************#
if not cp.restart:
    
    materialVariable.data[np.logical_and(sd < 0, sd > -1*ndp.mantleCrustDepth)] = crustID
    #materialVariable.data[np.logical_and(sd > 0,sd < ndp.mantleHarzDepth)] = harzID
#*************CHECKPOINT-BLOCK**************#
    

### surface

In [83]:
surfacexs = np.linspace(mesh.minCoord[0], mesh.maxCoord[0], md.res*md.aspectRatio*2)
surfaceys = np.ones(surfacexs.shape[0])
surfaceLine = markerLine2D(mesh, velocityField,surfacexs, surfaceys , ndp.faultThickness,  2)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


In [84]:
#variables for surface

surfVelx = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
surfStrain = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
surfVisc = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
surfTgrad = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
surfPres = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)


#variables for mid-lithsphere (saved on surface swarm)
midlithEvalEvalPoints = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 2)
midlithVisc = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
midlithSr2Inv = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
midlithVelX = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
midlithStrainTens = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 3)

#variables for upper-lithsphere (saved on surface swarm)
uplithEvalEvalPoints = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 2)
uplithVisc = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
uplithSr2Inv = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
uplithVelX = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 1)
uplithStrainTens = uw.swarm.SwarmVariable(surfaceLine.swarm, 'double', 3)

### fault

In [85]:
#the fault swarm will be positioned at faultLoc distance below the top of the slab, 
#intended to be between 0. and faultThickness
#md.faultLocFac=0.

faultLoc = md.faultLocFac*ndp.faultThickness

In [86]:
#parallel safety
if marker.swarm.particleCoordinates.data.shape[0]:
    ds_depth = max(0.0, abs(faultLoc - ndp.faultThickness))
    faultPoints = marker.swarm.particleCoordinates.data - marker.director.data[...]*ds_depth
    faultPoints[:,1][faultPoints[:,1] >= 1.0] = 0.999999 #make sure no fault point have leaked through the top
else:
    faultPoints = np.array([[999999.,999999.]])

#ds = ndp.faultThickness
#parallel safety
if marker.swarm.particleCoordinates.data.shape[0]:
    faultPoints = marker.swarm.particleCoordinates.data.copy()
else:
    faultPoints = np.array([[999999.,999999.]])

In [87]:
#fault thickness is thickness either side of fault.swarm

fault = fault2D(mesh, velocityField, [], [], 
                ndp.faultThickness, ndp.frictionInterface, ndp.cohesionInterface, 1, insidePt=(-0.5,0.5))

#*************CHECKPOINT-BLOCK**************#
cp.addObject(fault.swarm,'fault')

if not cp.restart:
    if faultPoints.shape[0]:
        fault.add_points(faultPoints[:, 0], faultPoints[:, 1])

else:
    fault.swarm.load(cp.loadpath + '/fault.h5')
#*************CHECKPOINT-BLOCK**************#
        
fault.rebuild()
fault.swarm.update_particle_owners()
    
    
fault_coll = fault_collection([fault])

In [88]:
#for the initial condition


def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


In [89]:
def update_swarm_from_faults(faults, proximityVariable):
    """
    Compute fault attributes from the marker-line objects in the 'faults' list.
    Specifically:
    
      - proximityVariable carries information about which fault each swarm particle is close to (0 means none)
      - normalVectorVariable maps the orientation of the fault to nearby swarm particles
      - signedDistanceVariable carries the distance (positive means 'inside')  
      
      Unchecked error: if these variables are from different swarms 
      
    """
    
    for fault_seg in faults:

        swarm = proximityVariable.swarm
                
        sd, pts0 = fault_seg.compute_signed_distance(swarm.particleCoordinates.data, distance=3.*fault_seg.thickness)
        
        effDist = 1. + 2.*gaussian( swarm.particleCoordinates.data[:,0], subZoneLoc.value + 0.5*thicknessAtTrench, thicknessAtTrench)
        distMultiplier = 1./effDist
        #insulate processors that have no fault
        if sd.shape[0]:
            sd[:,0]*=distMultiplier
        

        
        proximityVariable.data[np.logical_and(sd<(ndp.faultThickness - faultLoc),sd>-1.*faultLoc)] = 1
        
    
    return

In [90]:
#proximityVariable.data[:] = 0.0
#build the proximity 
if not cp.restart:
    update_swarm_from_faults(fault_coll, proximityVariable)

In [91]:
# Setup a swarm to define the replacment positions

allxs = np.arange(mesh.minCoord[0], mesh.maxCoord[0], ds_fault)[:-1]
allys = (mesh.maxCoord[1] - faultLoc)*np.ones(allxs.shape)

swarmTracerTop = uw.swarm.Swarm( mesh=mesh )
dummy =  swarmTracerTop.add_particles_with_coordinates(np.column_stack((allxs, allys)))
del allxs
del allys

In [92]:
def update_swarm_from_faults(faults, proximityVariable):
    """
    Compute fault attributes from the marker-line objects in the 'faults' list.
    Specifically:
    
      - proximityVariable carries information about which fault each swarm particle is close to (0 means none)
      - normalVectorVariable maps the orientation of the fault to nearby swarm particles
      - signedDistanceVariable carries the distance (positive means 'inside')  
      
      Unchecked error: if these variables are from different swarms 
      
    """
    
    for fault_seg in faults:

        swarm = proximityVariable.swarm
        
        #swarm = proximityVariable.swarm
        sd, pts0 = fault_seg.compute_signed_distance(swarm.particleCoordinates.data, distance=3.*fault_seg.thickness)
        #sp, pts0 = fault_seg.compute_marker_proximity(swarm.particleCoordinates.data)
        
        
        proximityVariable.data[np.logical_and(sd<(ndp.faultThickness - faultLoc),sd>-1.*faultLoc)] = 1
        
        
        #upper limit 
        faultUpLimFac = 2.0
        proximityVariable.data[np.logical_or(sd>faultUpLimFac*(ndp.faultThickness - faultLoc),sd<-faultUpLimFac*faultLoc)] = 0

    
    return

In [93]:
#update_swarm_from_faults(fault_coll, proximityVariable)

In [94]:
#fig= glucifer.Figure(quality=3)

#fig.append( glucifer.objects.Surface(mesh, temperatureField ))
#fig.append( glucifer.objects.Points(swarmTracerTop, pointSize=2))
#fig.append( glucifer.objects.Points(swarm, proxyTempVariable,pointSize=2))
#fig.append( glucifer.objects.Points(fault.swarm, pointSize=4))

#fig.append( glucifer.objects.Points(slabLine.swarm, pointSize=4))
#fig.append( glucifer.objects.Mesh(mesh))
##
#fig.show()
#fig.save_database('test.gldb')

## Interpolate to temperature field

In [95]:
def swarmToTemp():

    _ix, _weights, _dist = nn_evaluation(swarm.particleCoordinates.data, mesh.data, n=4, weighted=True)


    #_dist.shape, mesh.data.shape
    #if 
    tempMapTol = 0.2
    tempMapMask = _dist.min(axis=1) < tempMapTol*(1. - mesh.minCoord[1])/mesh.elementRes[1] 
    
    #temperatureField.data[:] = 0.
    temperatureField.data[:] = ndp.potentialTemp #first set to dimensionless potential temp

    #now used IDW to assign temp from particles to Field
    #this is looking pretty ugly; nn_evaluation could use some grooming
    temperatureField.data[:,0][tempMapMask] = np.average(proxyTempVariable.evaluate(swarm)[_ix][tempMapMask][:,:,0],weights=_weights[tempMapMask], axis=1)

    #now cleanup any values that have fallen outside the Bcs

    temperatureField.data[temperatureField.data > 1.] = ndp.potentialTemp
    temperatureField.data[temperatureField.data < 0.] = 0.
    
    #and cleanup the BCs
    
    temperatureField.data[bWalls.data] = ndp.potentialTemp
    temperatureField.data[tWalls.data] = 0.
    
    #if we're not imposing the rigdes, we want to leave these alone
    if md.fixedRidges:
        temperatureField.data[iWalls.data] = ndp.potentialTemp


In [96]:
#map proxy temp (swarm var) to mesh variable

if not cp.restart:
    swarmToTemp()

In [97]:
#fig= glucifer.Figure(quality=3, boundingBox= bBox)

#fig.append( glucifer.objects.Surface(mesh, slabCirc ))
#fig.append( glucifer.objects.Points(swarm, proximityVariable, pointSize=2))
#fig.append( glucifer.objects.Points(marker.swarm, pointSize=4))

#
#fig.show()
#fig.save_database('test.gldb')

In [98]:
#np.unique(proximityVariable.data)

In [99]:
#ndp.tempGradMantle
#ndp.tempGradMantle
#tempAtTrench 

In [100]:
#fig= glucifer.Figure(quality=3, boundingBox= bBox)

#fig.append( glucifer.objects.Mesh(mesh ))
#fig.append( glucifer.objects.Points(marker.swarm, pointSize=2))
#fig.append( glucifer.objects.Points(slabLine.swarm, pointSize=2))

#fig.append( glucifer.objects.Points(swarm, proxyTempVariable, pointSize=1))

#fig.show()
#fig.save_database('test.gldb')

## Swarm densification

Try to build our initial geometry with less particles that are required dynamically, then use population_control to fill out the swarm

In [101]:
population_control = uw.swarm.PopulationControl(swarm, deleteThreshold=0.006, splitThreshold=0.25, maxDeletions=1, maxSplits=3, aggressive=True,aggressiveThreshold=0.9, particlesPerCell=int(md.ppc))


#

In [102]:
def repopulate():
    thresh = 5000.
    diff = thresh + 1
    count = 1
    maxLoops = 10
    pg = np.copy(swarm.particleGlobalCount)
    while abs(diff) > thresh and count < maxLoops + 1:
        population_control.repopulate()
        diff = swarm.particleGlobalCount - pg
        pg = swarm.particleGlobalCount
        #print(str(count), str(pg), str(diff))
        count += 1
    

In [103]:
#run repop function
repopulate()

In [104]:
#((float(swarm.particleGlobalCount)/mesh.elementsGlobal))/md.ppc

## age variable


In [105]:
#We'll also create a field to occompany the swarm var
lithAgeField = uw.mesh.MeshVariable(mesh,nodeDofCount=1)

In [106]:
#Now we'll set to zero the values, that aren't in the thermal lithosphere

if not cp.restart:
    ageVariable.data[:] =  proxyageFn.evaluate(swarm)

dt_init = 0.


lithAgeConds = fn.branching.conditional([(temperatureField < 0.9, (ageVariable + dt_init)), #idea is to make this arbitrarily complex
                                         (True, 0.) ])



ageVariable.data[:] = lithAgeConds.evaluate(swarm)

del dt_init

#Now define an minimum age for crust creation
#ageCrustCreate = (5e6*(3600*24*365.))/sf.time
ageCrustCreate = 0.1*ndp.slabMaxAge

In [107]:
ix1, weights1, d1 = nn_evaluation(swarm.particleCoordinates.data, mesh.data, n=5, weighted=True)
    
    
#populate the mesh var. with an IDW interp.
lithAgeField.data[:,0] =  np.average(ageVariable.evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)


In [108]:
#fig= glucifer.Figure(quality=3, boundingBox= bBox)
#fig.append( glucifer.objects.Surface(mesh,lithAgeField > ageCrustCreate))
#fig.append( glucifer.objects.Points(swarm, materialVariable, pointSize=2))
##fig.append( glucifer.objects.Points(surfaceLine.swarm, pointSize=2))

#fig.show()
#fig.save_database('test.gldb')

In [109]:
#materialVariable.data[:] = 0.

##  Material Graph

In [110]:
###################
#initial particle layout
###################

#Setup the graph object
MG = MatGraph()

#First thing to do is to add all the material types to the graph (i.e add nodes)
MG.add_nodes_from(material_list)

#mantle  => crust
MG.add_transition((mantleID,crustID), depthFn, operator.lt, ndp.mantleCrustDepth)
MG.add_transition((mantleID,crustID), lithAgeField, operator.gt, ageCrustCreate) #avoid crust near ridges
MG.add_transition((mantleID,crustID), xFn, operator.lt, ndp.subZoneLoc)
MG.add_transition((mantleID,crustID), xFn, operator.gt, mesh.minCoord[0]*0.8) #avoid crust near wall


#crust  => mantle                
MG.add_transition((crustID, mantleID), depthFn, operator.gt, ndp.interfaceDestroyDepth)

MG.build_condition_list(materialVariable)

if not cp.restart:
    materialVariable.data[:] = fn.branching.conditional(MG.condition_list).evaluate(swarm)

In [111]:
#Final particle transformation rules
#restrict crust creation - avoid crust on the upper plate

MG.remove_edges_from([(mantleID,crustID)])

#mantle  => crust
MG.add_transition((mantleID,crustID), depthFn, operator.lt, ndp.mantleCrustDepth)
MG.add_transition((mantleID,crustID), lithAgeField, operator.gt, ageCrustCreate)
MG.add_transition((mantleID,crustID), xFn, operator.lt, (mesh.minCoord[0] - ndp.subZoneLoc)/2.)
MG.add_transition((mantleID,crustID), xFn, operator.gt, mesh.minCoord[0]*0.8)

MG.build_condition_list(materialVariable)

## choose temp field to use

In [112]:
if md.thermal:
    temperatureFn = temperatureField
else:
    temperatureFn = proxyTempVariable
    
#This is not handled entirely consistnetly from here... need to rethink this

#rebuild functions / clarify dependencies

if md.thermal:
    lithAgeConds = fn.branching.conditional([(temperatureFn < 0.9, (ageVariable)), #idea is to make this arbitrarily complex
                                         (True, 0.) ])


## adiabatic temp correction

In [113]:
#Adiabatic correction: this is added to the arrhenius laws to simulate the adiabatic component
#We'll use a double linearisation of the adiabatic temp function:

#ndp.tempGradMantle linearised at the mantle potential temp
#dp.tempGradSlab linearised at typical slab temp


tempAtTrench  = ndp.potentialTemp*math.erf((thicknessAtTrench)/(2.*math.sqrt(1.*ndp.slabMaxAge))) 
#this is the isotherm used to define the slab / mantle boundary, should be 0.9*Tp,


adiabaticCorrectFn = fn.branching.conditional([(temperatureFn> tempAtTrench, depthFn*ndp.tempGradMantle), #idea is to make this arbitrarily complex
                                      (True, depthFn*ndp.tempGradSlab) ])



In [114]:
## Also define the approx temp in C, for convenience 
tempCent = (temperatureFn+ adiabaticCorrectFn)*sf.deltaTemp



## Rheology

In [115]:

symStrainrate = fn.tensor.symmetric( 
                            velocityField.fn_gradient )

#Set up any functions required by the rheology
strainRate_2ndInvariant = fn.tensor.second_invariant( 
                            fn.tensor.symmetric( 
                            velocityField.fn_gradient ))



def safe_visc(func, viscmin=ndp.viscosityMin, viscmax=ndp.viscosityMax):
    return fn.misc.max(viscmin, fn.misc.min(viscmax, func))



#Add some portion of dynamic pressure to the depth-dependent Yield function
#dynamicPressureProxyDepthFn = pressureField/ndp.rayleigh
dynamicPressureProxyDepthFn = pressureField/sf.pressureDepthGrad

druckerDepthFn = fn.misc.max(0.0, depthFn + md.druckerAlpha*(dynamicPressureProxyDepthFn))
druckerFaultDepthFn = fn.misc.max(0.0, depthFn + md.druckerAlphaFault*(dynamicPressureProxyDepthFn))

In [116]:
#Diffusion Creep
diffusionUM = (1./ndp.diffusionPreExp)*\
            fn.math.exp( ((ndp.diffusionEnergy + (depthFn*ndp.diffusionVolume))/((temperatureFn+ adiabaticCorrectFn + ndp.surfaceTemp))))

diffusionLM = ndp.lowerMantleViscFac*(1./ndp.lowerMantlePreExp)*\
            fn.math.exp( ((ndp.lowerMantleEnergy + (depthFn*ndp.lowerMantleVolume))/((temperatureFn+ adiabaticCorrectFn + ndp.surfaceTemp))))

viscosityLM = safe_visc(diffusionLM)

#Add non-linearity
viscosityUM0 = safe_visc(diffusionUM)

if md.powerLaw:
    powerLawSRFn= ((strainRate_2ndInvariant+ 1e-15)/ndp.powerLawStrain)**((1.-ndp.powerLawExp)/ndp.powerLawExp)
    viscPower = viscosityUM0*powerLawSRFn
    effviscosity = viscPower*viscosityUM0/(viscPower + viscosityUM0) #then combine harmonically
    viscosityUM = safe_visc(effviscosity)

else:
    viscosityUM = viscosityUM0
    
#combine upper an lower mantle   
mantleCreep = fn.branching.conditional( ((depthFn < ndp.lowerMantleDepth, viscosityUM ), 
                                           (True,                      diffusionLM )  ))


#Define the mantle Plasticity
ys =  ndp.cohesionMantle + (druckerDepthFn*ndp.frictionMantle)
ysf = fn.misc.min(ys, ndp.yieldStressMax)
yielding = ysf/(2.*(strainRate_2ndInvariant) + 1e-15) 


if md.viscCombine == 0:
    mantleRheologyFn = safe_visc(fn.misc.min(mantleCreep, yielding), viscmin=ndp.viscosityMin, viscmax=ndp.viscosityMax)
else:
    mantleRheologyFn =  safe_visc(mantleCreep*yielding/(mantleCreep + yielding), viscmin=ndp.viscosityMin, viscmax=ndp.viscosityMax)
    

In [117]:
normDepths = depthFn/ndp.refDepthInterface
interfaceCreep = ndp.refViscInterface*fn.math.exp(ndp.logDelVisc*(1. - normDepths) )
#interfaceCreep = safe_visc(interfaceCreep0, viscmin=ndp.viscosityMinInterface, viscmax= ndp.viscosityMaxInterface)

#we want these avail. in all cases, as we evaluate it.
interfaceys =  ndp.cohesionInterface + (druckerFaultDepthFn*ndp.frictionInterface)
interfaceysf = fn.misc.min(interfaceys, ndp.ysMaxInterface)


if ndp.viscosityMinInterface == ndp.viscosityMaxInterface: #isoviscous interface
    interfaceViscosityFn = ndp.viscosityMinInterface
    

elif md.plasticInterface: #pseudo-brittle interface
    interfaceYielding = interfaceysf/(2.*(strainRate_2ndInvariant) + 1e-15)
    #combine
    interfaceViscosityFn = safe_visc(fn.misc.min(interfaceCreep , interfaceYielding), viscmin=ndp.viscosityMinInterface, viscmax=ndp.viscosityMaxInterface)


else: # a linear, brittle equivalent visc implementation
    ndp.effStrainRate = ndp.subVelocity/ndp.faultThickness
    effStressUpper =  ndp.cohesionInterface + (depthFn*ndp.frictionInterface)
    interfaceYielding = effStressUpper/(2.*ndp.effStrainRate)
    #combine
    interfaceViscosityFn = safe_visc(fn.misc.min(interfaceCreep , interfaceYielding), viscmin=ndp.viscosityMinInterface, viscmax=ndp.viscosityMaxInterface)



depthTaperFn = cosine_taper(depthFn, ndp.interfaceViscCutoffDepth, ndp.interfaceViscEndWidth)
interfaceRheologyFn =  interfaceViscosityFn*(1. - depthTaperFn) + depthTaperFn*mantleRheologyFn

In [118]:
if md.interfaceType == 1:
    viscosityMapFn = fn.branching.map( fn_key = materialVariable,
                             mapping = {0:mantleRheologyFn,
                                        1:interfaceRheologyFn} )

elif md.interfaceType == 2:
    viscosityMapFn = fn.branching.map( fn_key = proximityVariable,
                             mapping = {0:mantleRheologyFn,
                                        1:interfaceRheologyFn} )


## Stokes vel

In [119]:
#viscMidMantle = diffusion.evaluate((ndp.leftLim + 1e-6, 1. - (200e3/sf.lengthScale)))
#velStokes = ((2./9)/viscMidMantle)*ndp.rayleigh*thicknessAtTrench**2
#fac = 100.*(365*24*3600)*(3.4482758620689656e-13)
#velStokes*fac

## thermal,  phase and compositional buoyancy


In [86]:
#Basic Thermal Density contrib.

thermalDensityFn = ndp.rayleigh*(1. - temperatureFn)


In [87]:
##############
#Set up phase density contributions (approach of Yuen and Christenson)
##############


adibaticTempFn = temperatureFn + adiabaticCorrectFn
buoyancyFactor = (dp.refGravity*sf.lengthScale**3)/(sf.viscosityScale*dp.refDiffusivity)
slopeFactor = (dp.deltaTemp/(dp.refDensity*dp.refGravity*sf.lengthScale))


#Scale the relavent dimensional parameters
olDepths= np.array([410e3,660e3])/sf.lengthScale                  #depths of phase transitions along adiabat
olTemps = (np.array([1600., 1900.]) - dp.surfaceTemp)/dp.deltaTemp   #temperatures of phase transitions along adiabat
olWidths = np.array([10e3, 20e3])/sf.lengthScale                  #width if transition
olClaps= np.array([2.5e6, -2.5e6])*slopeFactor             #Clapeyron slope of transition
olBuoyancies= np.array([180., 400.])*buoyancyFactor       #Scaled buoyancy contributions



#olivine
olivinePhase = phases(name = 'ol', 
                        depths=olDepths,                  #depths of phase transitions along adiabat
                        temps = olTemps,                  #temperatures of phase transitions along adiabat
                        widths = olWidths,                #width if transition
                        claps=olClaps,                    #Clapeyron slope of trnasition
                        buoyancies = olBuoyancies)        #density change of phase transition


olivine_phase_buoyancy = olivinePhase.buoyancy_sum(adibaticTempFn, depthFn)


#Pyroxene/garnet
#Scale the relavent dimensional parameters
grtDepths = np.array([60e3,400e3,720e3])/sf.lengthScale            #depths of phase transitions along adiabat
grtTemps = (np.array([ 1000., 1600., 1900.]) - dp.surfaceTemp)/dp.deltaTemp  #temperatures of phase transitions along adiabat
grtWidths = np.array([ 10e3, 20e3, 20e3])/sf.lengthScale                  #width of transition
grtClaps= np.array([ 0., 1.e6,  1e6])*slopeFactor             #Clapeyron slope of transition
grtBuoyancies= np.array([350, 150., 400.])*buoyancyFactor       #Scaled buoyancy contributions



garnetPhase = phases(name = 'grt', 
                        depths=grtDepths,                  #depths of phase transitions along adiabat
                        temps = grtTemps,                  #temperatures of phase transitions along adiabat
                        widths = grtWidths,                #width if transition
                        claps=grtClaps,                    #Clapeyron slope of trnasition
                        buoyancies = grtBuoyancies)        #density change of phase transition


garnet_phase_buoyancy = garnetPhase.buoyancy_sum(adibaticTempFn, depthFn)

In [88]:
##############
#Set up compositional density contributions
##############

olivineSurfDensity = 3240.
garnetSurfDensity =  3080.

olivine_comp_buoyancy = (dp.refDensity - olivineSurfDensity)*buoyancyFactor
garnet_comp_buoyancy = (dp.refDensity  - garnetSurfDensity)*buoyancyFactor

In [89]:
##############
#Combine the density contributions
##############

olivineInBasalt = 0.0
olivineInPyrolite = 0.6
olivineInHarzburgite = 0.8

#this is a crude way to adjust the buoyancies for models where the crust is thicker than physically accurate
crustWidthBuoyFac = 7.5e3/dp.mantleCrustDepth

basaltbuoyancyFn = thermalDensityFn  + \
                     crustWidthBuoyFac*(olivineInBasalt*(olivine_phase_buoyancy) + (1. - olivineInBasalt)*garnet_phase_buoyancy - \
                     olivineInBasalt* (olivine_comp_buoyancy) - (1. - olivineInBasalt)*garnet_comp_buoyancy)

pyrolitebuoyancyFn = thermalDensityFn  + \
                     olivineInPyrolite*(olivine_phase_buoyancy) + (1. - olivineInPyrolite)*garnet_phase_buoyancy - \
                     olivineInPyrolite* (olivine_comp_buoyancy) - (1. - olivineInPyrolite)*garnet_comp_buoyancy

harzbuoyancyFn = thermalDensityFn  + \
                     olivineInHarzburgite*(olivine_phase_buoyancy) + (1. - olivineInHarzburgite)*garnet_phase_buoyancy - \
                     olivineInHarzburgite* (olivine_comp_buoyancy) - (1. - olivineInHarzburgite)*garnet_comp_buoyancy

In [90]:
#md.phaseBuoyancy = True

In [91]:
if md.phaseBuoyancy:

    densityMapFn = fn.branching.map( fn_key = materialVariable,
                             mapping = {crustID:pyrolitebuoyancyFn, 
                                        mantleID:pyrolitebuoyancyFn} )

else:
    densityMapFn = fn.branching.map( fn_key = materialVariable,
                             mapping = {crustID:thermalDensityFn, 
                                        mantleID:thermalDensityFn} )

In [92]:
# Now create a buoyancy force vector using the density and the vertical unit vector. 

gravity = ( 0.0, -1.0 )

buoyancyMapFn = densityMapFn * gravity

## Stokes system and solver

In [93]:
print('got to Stokes')

got to Stokes


In [1]:
surfaceArea = uw.utils.Integral(fn=1.0,mesh=mesh, integrationType='surface', surfaceIndexSet=tWalls)
surfacePressureIntegral = uw.utils.Integral(fn=pressureField, mesh=mesh, integrationType='surface', surfaceIndexSet=tWalls)

NodePressure = uw.mesh.MeshVariable(mesh, nodeDofCount=1)
Cell2Nodes = uw.utils.MeshVariable_Projection(NodePressure, pressureField, type=0)
Nodes2Cell = uw.utils.MeshVariable_Projection(pressureField, NodePressure, type=0)

def smooth_pressure(mesh):
    # Smooths the pressure field.
    # Assuming that pressure lies on the submesh, do a cell -> nodes -> cell
    # projection.

    Cell2Nodes.solve()
    Nodes2Cell.solve()

# a callback function to calibrate the pressure - will pass to solver later
def pressure_calibrate():
    (area,) = surfaceArea.evaluate()
    (p0,) = surfacePressureIntegral.evaluate()
    offset = p0/area
    pressureField.data[:] -= offset
    smooth_pressure(mesh)

NameError: name 'uw' is not defined

In [97]:
stokesPIC = uw.systems.Stokes( velocityField  = velocityField, 
                                   pressureField  = pressureField,
                                   conditions     = [freeslipBC,],
                                   fn_viscosity   = viscosityMapFn, 
                                   fn_bodyforce   = buoyancyMapFn )



In [98]:
solver = uw.systems.Solver(stokesPIC)


if md.penaltyMethod:
    solver.set_inner_method("mumps")
    solver.options.scr.ksp_type="cg"
    solver.set_penalty(1.0e7)
    solver.options.scr.ksp_rtol = 1.0e-4

else:
    solver.options.main.Q22_pc_type='gkgdiag'
    solver.options.scr.ksp_rtol=5e-5
    solver.set_inner_method('mg')
    solver.options.mg.levels = 4
    
    
    
#avoid this solve if restarting
if not cp.restart:
    solver.solve(nonLinearIterate=True, nonLinearTolerance=md.nltol, callback_post_solve = pressure_calibrate)
    solver.print_stats()


 
Pressure iterations:   4
Velocity iterations:   1 (presolve)      
Velocity iterations:  -1 (pressure solve)
Velocity iterations:   1 (backsolve)     
Velocity iterations:   1 (total solve)   
 
SCR RHS  solve time: 6.1430e-01
Pressure solve time: 9.8819e-02
Velocity solve time: 7.5736e-01 (backsolve)
Total solve time   : 1.6038e+00
 
Velocity solution min/max: 0.0000e+00/0.0000e+00
Pressure solution min/max: 0.0000e+00/0.0000e+00
 



In [ ]:
#Now handled by the callback

#Define an integral to remove drift in pressure
#_pressure = surfint(pressureField)
#_surfLength = surfint()
#surfLength = _surfLength.evaluate()[0]

##pressureSurf = _pressure.evaluate()[0]   
#pressureField.data[:] -= pressureSurf/surfLength

In [ ]:
#fig= glucifer.Figure(quality=3)

#fig.append( glucifer.objects.Mesh(mesh, opacity=0.2))
#fig.append( glucifer.objects.Points(swarm, dynamicPressureProxyDepthFn + depthFn, pointSize=2))
#fig.append( glucifer.objects.Points(swarm, proxyTempVariable, pointSize=2))

#fig.append( glucifer.objects.Points(marker.swarm,  pointSize=2))

#fig.append( glucifer.objects.VectorArrows(mesh, velocityField, arrowHead=1, scaling=0.00005))
#fig.append( glucifer.objects.Surface(mesh,pyrolitebuoyancyFn ))
#fig.show()
#fig.save_database('test.gldb')

## Setup advection-diffusion, swarm advection

In [236]:
advector = uw.systems.SwarmAdvector( swarm=swarm, velocityField=velocityField, order=2 )



#test using temperatureFn, rather than field here (depends on how pointers work)
if md.thermal:
    advDiff = uw.systems.AdvectionDiffusion( phiField       = temperatureFn, 
                                         phiDotField    = temperatureDotField, 
                                         velocityField  = velocityField,
                                         fn_sourceTerm    = 0.0,
                                         fn_diffusivity = 1., 
                                         #conditions     = [neumannTempBC, dirichTempBC] )
                                         conditions     = [ dirichTempBC] )


## Initial thermal diffusion

Working, but not tuned or tested

In [98]:
if md.diffuseInitial and md.thermal:

    velocityField0   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
    velocityField0.data[:] = 0.0

    #testDepth = 50e3/sf.lengthScale
    #testWidth = 10e3/sf.lengthScale 

    #difftanhFn = 0.5*(fn.math.tanh((depthFn - testDepth)/(testWidth))) + 0.5


    advDiff0 = uw.systems.AdvectionDiffusion( phiField       = temperatureFn, 
                                             phiDotField    = temperatureDotField, 
                                             velocityField  = velocityField0,
                                             fn_sourceTerm    = 0.0,
                                             fn_diffusivity = 1.0, 
                                             #conditions     = [neumannTempBC, dirichTempBC] )
                                             conditions     = [ dirichTempBC] )




    diff_for  = md.diffuseInitial*(3600.*365.*24.)/sf.time
    dt_diff = 0.
    step_diff = 0.
    while dt_diff < diff_for:
        dt_now = advDiff0.get_max_dt()
        advDiff0.integrate(dt_now)
        dt_diff += dt_now
        step_diff += 1
        print('years and step: {} {}'.format((dt_diff*sf.time)/(3600.*365.*24.), step_diff))   



In [99]:
print('past first solve')

past first solve


## More functions and fields

For integration, querying, plotting, etc

In [237]:
#fields / mesh variables

stressFieldX    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
stressFieldMag    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
proximityField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
#restrictionFnField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
eig1       = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
eig2       = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )

In [238]:
#Setup any extra uw fucntions and handles to evaluate

sinner = fn.math.dot( strainRate_2ndInvariant, strainRate_2ndInvariant )

vd = 4.*viscosityMapFn*sinner #there's an extra factor of 2, which is necessary because the of factor of 0.5 in the UW second invariant 
vx = velocityField[0]

#density function here is okay, as |g| = 1
dw = thermalDensityFn*velocityField[1] #if compositional buoyancy is added, we'll need to add the the proper Buoyancy Fn


sym_strainRate = fn.tensor.symmetric( 
                            velocityField.fn_gradient )

stressXVariableFn =  2.*sym_strainRate[0]*viscosityMapFn
stressMagVariableFn =  2.*strainRate_2ndInvariant*viscosityMapFn

if md.thermal:
    dTdZ = temperatureFn.fn_gradient[1]
else:
    dTdZ = fn.misc.constant(1.)

## Values and metrics

Mostly, this consists of integral and min/max values. 

The most challenging part is defining restricted domains for these calculations.





In [240]:
#Setup integral domain / restriction functions


#0 - these ones are used in tracking the movement of plates Tres Importantee
#at each step, out plate boundary search will happen in these limits only
pbbound = 150e3/sf.lengthScale
pbwidth = 10e3/sf.lengthScale
szRfn150 = 1. - cosine_taper(fn.math.abs(xFn - subZoneLoc),pbbound, pbwidth)
szRfn300 = 1. - cosine_taper(fn.math.abs(xFn - subZoneLoc),2*pbbound, pbwidth)

spRidgeRfn = 1. - cosine_taper(fn.math.abs(xFn - spRidgeLoc),pbbound, pbwidth)
opRidgeRfn = 1. - cosine_taper(fn.math.abs(xFn - opRidgeLoc),pbbound, pbwidth)

#1
#Level 3. thermal lithosphere - mantle:
lithconditions = [ (         temperatureFn < 0.9,  1.),
                   (                                                   True , 0.) ]
lithTempRFn = fn.branching.conditional(lithconditions)

#2 
upMantleconditions = [ (         depthFn < ndp.lowerMantleDepth,  1.),
                   (                                                   True , 0.) ]

upMantleRfn = fn.branching.conditional(upMantleconditions)

#3
lowMantleconditions = [ (         depthFn > ndp.lowerMantleDepth,  1.),
                   (                                                   True , 0.) ]

lowMantleRfn = fn.branching.conditional(lowMantleconditions)

#5

MantleSlabConditions = [ (operator.and_((lithTempRFn > 0.5),(depthFn > ndp.lowerMantleDepth)) , 1.),
                        (                                                   True , 0.) ]

lowMantleSlabRfn = fn.branching.conditional(MantleSlabConditions)

#5

#also add RMS for the lithosphere, or some measure of relative subduction rate
velLimitX = 1./50. #these will need tuning!
velLimitY = 1./50.
velXNorm = velocityField[0]/ndp.subVelocity
velYNorm = velocityField[1]/ndp.subVelocity

lowPlateConditions = [ (           operator.and_( operator.and_((depthFn < thicknessAtTrench), (lithTempRFn > 0.5)),  (velXNorm> -1.0*velLimitX)), 1.),
                       (operator.and_((lithTempRFn > 0.5),(velYNorm < -1*velLimitY))  , 1.),
                       (operator.and_((depthFn > thicknessAtTrench), (lithTempRFn > 0.5)),1.),
                       (                                                   True , 0.) ]

lowPlateRfn = fn.branching.conditional(lowPlateConditions)


#6

lowPlateBendingConds = [ (operator.and_((lowPlateRfn > 0.5),(depthFn < (300e3/sf.lengthScale))) , 1.),
                        (                                                   True , 0.) ]

lowPlateBendingRfn = fn.branching.conditional(lowPlateBendingConds)



#7
if md.interfaceType == 2:
    interfaceConds = [ (operator.and_((proximityVariable > 0.9), (proximityVariable < 1.1)), 1.),
                        (                                                   True , 0.) ]
    interfaceRfn0 = fn.branching.conditional(interfaceConds)
else:

    interfaceConds = [ (operator.and_((materialVariable > 0.9), (materialVariable < 1.1)), 1.),
                        (                                                   True , 0.) ]
    interfaceRfn0 = fn.branching.conditional(interfaceConds)

#also limit the depth for the interface integrals
interfaceDepthLimitConds = [ (depthFn < 150e3, 1.),
                        (                                                   True , 0.) ]
interfaceDepthLimitFn = fn.branching.conditional(interfaceDepthLimitConds)
 
interfaceRfn = interfaceDepthLimitFn*interfaceRfn0

#8 , a function that should exclude all mantle except the wedge
#This is based on a straight line that starts 50 km arcward of the trench, and with a slope of -0.75
#but should be flexible enough the majority of slab dips
wedgeConds = [ (depthFn > 250e3/sf.lengthScale, 0.0),
               (xFn > subZoneLoc + 400e3/sf.lengthScale, 0.0),
                                         ( coordinate[1] > 1. - 0.75*(xFn - (subZoneLoc + 50e3/sf.lengthScale)), 1.0),
                        (                                                   True , 0.) ]
wedgeRFn = fn.branching.conditional(wedgeConds)


#This is same domain as the function we use to evaluate the location of the hot corner
#This one has binary values intended for integration
critWedgeTempIntConds = [ (operator.and_(wedgeRFn > 0.5,tempCent > 1250. ), 1.0),
                        (                                                   True , 0.) ]
critWedgeTempIntFn = fn.branching.conditional(critWedgeTempIntConds)



#10 - these ones divide up the system domain into plates (assuming < 4 plate system)

#Always use operator! np.logical_and is not trustworthy when applied to uw functions
#spPlateConds = [ (    operator.and_( xFn >  spRidgeLoc, xFn <  subZoneLoc),  1.),
#                   (                                                   True , 0.) ]
#spPlateFn = fn.branching.conditional(spPlateConds)

#opPlateConds =  [ (    operator.and_( xFn >  subZoneLoc, xFn <  opRidgeLoc),  1.),
#                   (                                                   True , 0.) ]
#opPlateFn = fn.branching.conditional(opPlateConds)

#movingPlatesConds =  [ (    operator.and_( xFn >  spRidgeLoc, xFn <  opRidgeLoc),  1.),
#                   (                                                   True , 0.) ]
#movingPlatesFn = fn.branching.conditional(movingPlatesConds)


#sig = 15e3/sf.lengthScale #We'll use this to fix inf values that can occur at domain boundaries
#fixedridgeFn = 1. -  \
#                fn.math.exp(-1.*(xFn - mesh.minCoord[0])**2/(2 * sig**2))- \
#                fn.math.exp(-1.*(xFn - mesh.maxCoord[0])**2/(2 * sig**2))
    

In [241]:
#fig= glucifer.Figure(quality=3, boundingBox= bBox)
#fig.append( glucifer.objects.Surface(mesh, wedgeRFn))
#fig.append( glucifer.objects.Contours(mesh, temperatureFn ))
#fig.show()

In [242]:
#Setup integrals 

_dwint = volumeint(dw,1.)
_nuTop = surfint(dTdZ)


_tempint = volumeint(temperatureField)


_areaintcritWedgeTemp  = volumeint(critWedgeTempIntFn)

_areaintLith  = volumeint(lithTempRFn)
_vdintLith  = volumeint(vd,lithTempRFn)

_areaintLower  = volumeint(lowPlateRfn)
_vdintLower  = volumeint(vd,lowPlateRfn)

_areaintLowerBending  = volumeint(lowPlateBendingRfn)
_vdintLowerBending  = volumeint(vd,lowPlateBendingRfn)

#_areaintUM  = volumeint(upMantleRfn)
_vdintUM  = volumeint(vd,upMantleRfn)

#_areaintLM  = volumeint(lowMantleRfn)
_vdintLM  = volumeint(vd,lowMantleRfn)

_areaintSlabLM  = volumeint(lowMantleSlabRfn)
_vdintSlabLM  = volumeint(vd,lowMantleSlabRfn)

_areaintInterface  = volumeint(interfaceRfn)
_vdintInterface = volumeint(vd,interfaceRfn)

In [243]:
#'_mmVxSurfNearLeft'

In [244]:
#Setup max min fns 

#Surface extrema
_mmVxSurf = fn.view.min_max(vx)
dummyFn = _mmVxSurf.evaluate(tWalls)

#These ones are used to track plate boundaries
_szVelGrads = fn.view.min_max(sym_strainRate[0]*szRfn150, fn_auxiliary=xFn)
dummyFn = _szVelGrads.evaluate(tWalls)

_spRidgeVelGrads = fn.view.min_max(sym_strainRate[0]*spRidgeRfn, fn_auxiliary=xFn)
dummyFn = _spRidgeVelGrads.evaluate(tWalls)

_opRidgeVelGrads = fn.view.min_max(sym_strainRate[0]*opRidgeRfn, fn_auxiliary=xFn)
dummyFn = _opRidgeVelGrads.evaluate(tWalls)

#Misc

#this is used to build the location of the hot (meltable) mantle in wedge
critWedgeTempConds = [ (operator.and_(wedgeRFn > 0.5,tempCent > 1250. ), xFn),
                        (                                                   True , 10.) ]
critWedgeTempFn = fn.branching.conditional(critWedgeTempConds)

_wedgeHotCorner = fn.view.min_max(critWedgeTempFn, fn_auxiliary=coordinate)
dummyFn = _wedgeHotCorner.evaluate(mesh)



In [245]:
#import glucifer
#fig= glucifer.Figure(quality=3)
#
#fig.append( glucifer.objects.Surface(mesh, critWedgeTempFn ))
#fig.append( glucifer.objects.Points(swarm, proximityVariable, pointSize=2))
#fig.append( glucifer.objects.Points(marker.swarm, pointSize=4))
#fig.append( glucifer.objects.Points(fault.swarm, pointSize=4))
#fig.append( glucifer.objects.Mesh(mesh))
##
#fig.show()


# Viz

In [246]:
#Build a depth dependent mask for the vizualisation

depthVariable      = swarm.add_variable( dataType="float", count=1 )
depthVariable.data[:] = depthFn.evaluate(swarm)

vizVariable      = swarm.add_variable( dataType="int", count=1 )
vizVariable.data[:] = 0

for index, value in enumerate(depthVariable.data[:]):
    #print index, value
    if np.random.rand(1)**30 > value/(mesh.maxCoord[1] - mesh.minCoord[1]):
        vizVariable.data[index] = 1
    #if value > 660e3/sf.lengthScale:
    #    vizVariable.data[index] = 0
        
        
del index, value    #get rid of any variables that might be pointing at the .data handles (these are!)

#Now randomly cull more particles if desired

removeRandom = True
if removeRandom:
    reducFac = 0.6  #0.9 > remove 90%
    nonzs = np.where(vizVariable.data[:,0] == 1)[0].copy()
    nstart =  nonzs.shape[0]
    nend = int(np.ceil(nstart*reducFac))
    np.random.shuffle(nonzs)
    nstart, nend

    vizVariable.data[nonzs[:nend]] = 0
    del nonzs, nstart, nend



#Build any Functions we need for Viz

stressII = 2.*strainRate_2ndInvariant*viscosityMapFn
stressXX = 2.*symStrainrate[0]*viscosityMapFn




In [247]:
#Set up the gLucifer stores

fullpath = os.path.join(outputPath + "gldbs/")
store1 = glucifer.Store(fullpath + 'subduction1.gldb')
store2 = glucifer.Store(fullpath + 'subduction2.gldb')
store3 = glucifer.Store(fullpath + 'subduction3.gldb')
store4 = glucifer.Store(fullpath + 'subduction4.gldb')
store5 = glucifer.Store(fullpath + 'subduction5.gldb')
store6 = glucifer.Store(fullpath + 'subduction6.gldb')

fig1 = glucifer.Figure(store1,figsize=(300*np.round(md.aspectRatio,2),300))
if md.thermal:
    fig1.append( glucifer.objects.Points(swarm, temperatureFn, pointSize=2,  valueRange=[0.0, 1.0], fn_mask=vizVariable))
else:
    fig1.append( glucifer.objects.Points(swarm, proxyTempVariable, pointSize=2, valueRange=[0.0, 1.0],  fn_mask=vizVariable))
fig1.append( glucifer.objects.Points(fault.swarm, pointSize=5, colourBar=False))


fig2 = glucifer.Figure(store2,figsize=(300*np.round(md.aspectRatio,2),300))
fig2.append( glucifer.objects.Points(swarm, viscosityMapFn, pointSize=2, fn_mask=vizVariable, logScale=True, valueRange=[100.*ndp.viscosityMin, ndp.viscosityMax]))
fig2.append( glucifer.objects.VectorArrows(mesh, velocityField, arrowHead=1, scaling=0.00005))


fig3 = glucifer.Figure(store3,figsize=(300*np.round(md.aspectRatio,2),300))
fig3.append( glucifer.objects.Points(swarm, pressureField, pointSize=2, fn_mask=vizVariable))
fig3.append( glucifer.objects.Mesh(mesh))

fig3.append( glucifer.objects.Points(marker.swarm, pointSize=5, colourBar=False))


fig4 = glucifer.Figure(store4,figsize=(300*np.round(md.aspectRatio,2),300))
fig4.append( glucifer.objects.Surface(mesh, lithAgeField ))
fig4.append( glucifer.objects.Surface(mesh, critWedgeTempFn))
fig4.append( glucifer.objects.Surface(mesh, szRfn150))


if md.thermal:
    fig4.append( glucifer.objects.Surface(mesh, lowPlateRfn ))
fig4.append( glucifer.objects.Points(fault.swarm, pointSize=5, colourBar=False))





fig5 = glucifer.Figure(store5,figsize=(300*np.round(md.aspectRatio,2),300))
fig5.append( glucifer.objects.Points(swarm, stressII , pointSize=2, fn_mask=vizVariable, valueRange=[0.0,ndp.yieldStressMax]))
fig5.append( glucifer.objects.Points(swarm, stressXX , pointSize=2, fn_mask=vizVariable, valueRange=[-1*ndp.yieldStressMax/2.,ndp.yieldStressMax/2.]))
#fig5.append(glucifer.objects.Contours(mesh, temperatureField))

fig6 = glucifer.Figure(store6,figsize=(300*np.round(md.aspectRatio,2),300))
if md.interfaceType==2:
    fig6.append( glucifer.objects.Points(swarm, proximityVariable, pointSize=2, fn_mask=vizVariable))
else:
    fig6.append( glucifer.objects.Points(swarm, materialVariable, pointSize=2, fn_mask=vizVariable))
fig6.append( glucifer.objects.Points(fault.swarm, pointSize=5, colourBar=False))
fig6.append( glucifer.objects.Points(swarm, materialVariable, pointSize=2, fn_mask=vizVariable))

## Update functions for main loop

In [248]:
def main_update(next_image_step):
    
    """
    This includes some functionality for image / file writing at speciified time intervals,
    Assumes global variables:
        time, step, files_freq, next_image_step
    
    if numerical dt exceeds next specified writing point
    override dt make sure we hit that point
    Set some flags so that image / file writing proceeds
    
    """
    
    
    if md.thermal:
        dt = advDiff.get_max_dt()*md.courantFac #additional md.courantFac helps stabilise advDiff
        advDiff.integrate(dt)
        
    else:
        dt = advector.get_max_dt()
        
    #This relates to file writing at set period:
    #override dt make sure we hit certain time values
    #Set some flags so that image / file writing proceeds
    
    if step == 0:
        files_this_step = True
    else:
        files_this_step = False
    
    if time + dt >= next_image_step:
        dt = next_image_step - time
        files_this_step = True
        next_image_step += files_freq #increment time for our next image / file dump
        
        
    #Do advection
    #I inlude marker swarms here, so their tailored update functions don't havre to be run every time step
    
    advector.integrate(dt)
    marker.advection(dt)
    fault.advection(dt)
    
    #Now handled by the solver callback
    #remove drift in pressure
    #pressureSurf = _pressure.evaluate()[0]   
    #pressureField.data[:] -= pressureSurf/surfLength
    
    
    return time+dt, step+1,dt, files_this_step, next_image_step
    

In [271]:
dataToadd = np.empty((0,2)).astype(int) #a dummy array
    
    
#Add any particles that remain
fault.swarm.add_particles_with_coordinates(dataToadd)

array([], dtype=int32)

In [272]:
def fault_update():
    
    """
    Assumes global variables:
    
    * dt
    * ndp.interfaceDestroyDepth
    * sf.lengthScale
    * lithAgeField
    * ds_fault             #fault particle spacing
    * md.swarmUpdate
    * subZoneLoc (not ndp.subZoneLoc)
    
    """
    
    #reset to zero
    #proximityVariable.data[:,:] = 0.0
  
    #First, we'll try to add particles back in...
    #a number of constraints are placed on which ones we choose
    
    #avoid particles which already hit the proximity region (field)
    #newMask1 = np.where(
    #    proximityField.evaluate(np.column_stack(
    #        (swarmTracerTop.particleCoordinates.data[:,0], swarmTracerTop.particleCoordinates.data[:,1] + 0.49*faultLoc ))
    #                           )[:,0] == 0)[0]
    
    #remove particles in prohibited regions
   
    if swarmTracerTop.particleCoordinates.data.shape[0]:
        newMask1 = np.where(swarmTracerTop.particleCoordinates.data[:,0] < (subZoneLoc.value - 150e3/sf.lengthScale))[0]
    
        #remove particles that are nearby to existing ones (ds_fault is the expected distance between fault points)
        if not fault.empty:
            ds, ids =  fault.kdtree.query(swarmTracerTop.particleCoordinates.data)
            #This should not aim to to perform regular partical addition, hence factor of 4
            #otherwise, can two horizontal layers forming
            newMask2 = np.where(ds > 4*ds_fault)[0]

            totesMask = np.intersect1d(newMask1, newMask2)
        #if there is no fault locally, we don't need to check
        else:
            totesMask = newMask1

        dataToadd = swarmTracerTop.particleCoordinates.data[totesMask, :]

    else:
        dataToadd = np.empty((0,2)).astype(int) #a dummy array
    
    #Add any particles that remain
    fault.swarm.add_particles_with_coordinates(dataToadd)
    
    #cull any below cutoff depth
    cutoffDepth = ndp.interfaceViscCutoffDepth + ndp.interfaceViscEndWidth
    depthMask1 = np.where(fault.swarm.particleCoordinates.data[:,1] < (mesh.maxCoord[1] - cutoffDepth))[0]
    
    with fault.swarm.deform_swarm():
        fault.swarm.particleCoordinates.data[depthMask1] = (999999.,999999.)
    
    uw.barrier()
    
    #and any particles where the age is too young - aimed at reducing the fault interacting with the MOR.
    if fault.swarm.particleCoordinates.data.shape[0]:
        ageMask = np.where(lithAgeField.evaluate(fault.swarm) < ageCrustCreate)[0]
    else:
        ageMask = np.empty(0, dtype=np.bool)
    with fault.swarm.deform_swarm():
        fault.swarm.particleCoordinates.data[ageMask] = (999999.,999999.)
        
        
    #Reset any particles beneath interface reset depth
    #i.e. we don't let proximity 'accumulate beneath this depth'
    
    depthMask2 = np.where(swarm.particleCoordinates.data[:,1] < (1. - ndp.interfaceResetDepth))
    proximityVariable.data[depthMask2] = 0
        
    
    #now we can rebuild the fault
    fault.rebuild()
    update_swarm_from_faults(fault_coll, proximityVariable)
    
    #for any particles deeper than ndp.interfaceDestroyDepth, proximity goes to zero as well
    
    depthMask3 = np.where(swarm.particleCoordinates.data[:,1] < (1. - ndp.interfaceDestroyDepth))
    proximityVariable.data[depthMask3] = 0
    

In [273]:
#dt = 0.00001
#fault_update()

In [116]:
def age_update():
    
    """
    update age variable which requires some special attention
    Assumes global variables:
    
    * dt
    * step 
    """
    
    #this one run whether the model is thermal or not
    # (although the lithAgeField will have less meaning for the compositional model )
    
    #then we increment the age everwhere
    ageVariable.data[:]+=dt
    
    #occasionally reset
    if md.thermal:
        if step%10==0:
            ageVariable.data[:] = lithAgeConds.evaluate(swarm)

In [117]:
def viz_update():
    
    #Rebuild the viz. mask
    vizVariable.data[:] = 0

    for index, value in enumerate(depthVariable.data[:]):
        #print index, value
        if np.random.rand(1)**5 > value/(mesh.maxCoord[1] - mesh.minCoord[1]):
            vizVariable.data[index] = 1

    del index, value    #get rid of any variables that might be pointing at the .data handles (these 
    
    if removeRandom:
        reducFac = 0.6  #0.9 > remove 90%
        nonzs = np.where(vizVariable.data[:,0] == 1)[0].copy()
        nstart =  nonzs.shape[0]
        nend = int(np.ceil(nstart*reducFac))
        np.random.shuffle(nonzs)
        nstart, nend

        vizVariable.data[nonzs[:nend]] = 0
        del nonzs, nstart, nend   
    
    #save gldbs
    fullpath = os.path.join(outputPath + "gldbs/")
    
    store1.step = step
    fig1.save( fullpath + "Temp" + str(step).zfill(5))
    
    store2.step = step
    fig2.save( fullpath + "visc" + str(step).zfill(5))
    
    #store3.step = step
    #fig3.save( fullpath + "pressure" + str(step).zfill(5))
    
    store4.step = step
    fig4.save( fullpath + "age_lith" + str(step).zfill(5))
    
    #store5.step = step
    #fig5.save( fullpath + "strainrate" + str(step).zfill(5))
    
    store6.step = step
    fig6.save( fullpath + "strainrate" + str(step).zfill(5))
    

In [118]:
def swarm_update():
    
    #run swarm repopulation
    repopulate()
  
    
    #rebuild the material graph condition list, and apply to swarm
    MG.build_condition_list(materialVariable)
    materialVariable.data[:] = fn.branching.conditional(MG.condition_list).evaluate(swarm)
   

In [119]:
def markerLine_update():
    cutoffDepth = 400e3/sf.lengthScale
    mask = marker.swarm.particleCoordinates.data[:,1] < (mesh.maxCoord[1] - cutoffDepth)

    with marker.swarm.deform_swarm():
        marker.swarm.particleCoordinates.data[mask] = (999999.,999999.)
        


In [101]:
_lambda = 0.5      #A dampening  applied to teh entire laplace vector
laplaceLimit = 0.25 #fraction of inter-particle distance that is the maximum laplace displacement

def heal_markers_update(smoothCycles, k):
    
    """
    Assumes: ds_fault
    """
    
    ###########
    #Marker
    ###########
    
    #Smoothing
    for cyc in range(smoothCycles):
        marker.rebuild()
        if not marker.empty:
            Dl = laplaceVector(marker, k = k, limit=laplaceLimit)
        else:
            Dl = 0.0  

        with marker.swarm.deform_swarm():
                marker.swarm.particleCoordinates.data[:] -= _lambda *Dl
    
    
    #Addition
    marker.rebuild()
    if not marker.empty:
        A = marker.neighbourMatrix( k =k)
        newPoints = particlesToAdd(marker, A, _lowdist=2.*ds_fault) #start addding particles above _lowdist
    else:
        newPoints = np.empty((0,2))
    marker.add_points(newPoints[:,0], newPoints[:,1])
    
    #Removal
    #dummy arrays to use in case there's no marker on the proc 
    marker.rebuild()
    midPoints = np.empty((0,2))
    currentIds = np.empty((0,)).astype('bool') 
    if not marker.empty:
        A = marker.neighbourMatrix( k =k)
        midPoints, currentIds = neighbourDistanceQuery(marker, A, _lowdist=1e-6*ds_fault,_updist= 0.5*ds_fault)
        #Need to delete those points first, before the 
    with marker.swarm.deform_swarm():
        marker.swarm.particleCoordinates.data[currentIds] = (9999999., 9999999.)
            
    marker.add_points(midPoints[:,0], midPoints[:,1])
    
    
    ###########
    #Fault
    ###########
    
    #Smoothing
    for cyc in range(smoothCycles):
        fault.rebuild()
        if not fault.empty:
            Dl = laplaceVector(fault, k = k, limit=laplaceLimit)
        else:
            Dl = 0.0  

        with fault.swarm.deform_swarm():
                fault.swarm.particleCoordinates.data[:] -= _lambda *Dl
    
    
    #Addition
    fault.rebuild()
    if not fault.empty:
        A = fault.neighbourMatrix( k =k)
        newPoints = particlesToAdd(fault, A, _lowdist=2.*ds_fault)
    else:
        newPoints = np.empty((0,2))
    fault.add_points(newPoints[:,0], newPoints[:,1])
    
    #Removal
    fault.rebuild()
    #dummy arrays to use in case there's no marker on the proc 
    midPoints = np.empty((0,2))
    currentIds = np.empty((0,)).astype('bool')     
    if not fault.empty:
        A = fault.neighbourMatrix( k =k)
        midPoints, currentIds = neighbourDistanceQuery(fault, A, _lowdist=1e-6*ds_fault,_updist= 0.5*ds_fault)
        #Need to delete those points first, before the 
    with fault.swarm.deform_swarm():
        fault.swarm.particleCoordinates.data[currentIds] = (9999999., 9999999.)
            
    fault.add_points(midPoints[:,0], midPoints[:,1])
    

In [102]:
#heal_markers(1, 4)

In [120]:
def pressure_contour_points_update():
    
    criticalTempConds =  fn.branching.conditional([(operator.and_(proxyTempVariable > 0.9, proxyTempVariable < 0.98), True), 
                                      (True, False) ])

    mask = criticalTempConds.evaluate(swarm)[:,0]
    pointsRetain = swarm.particleCoordinates.data[mask,:]

    pressureContourSwarm = uw.swarm.Swarm( mesh=mesh )
    pressureContourSwarm.add_particles_with_coordinates(pointsRetain)
    pressureOnContour = pressureContourSwarm.add_variable( dataType="double", count=1 )
    pressureOnContour.data[:] = pressureField.evaluate(pressureContourSwarm)
    
    fullpath = os.path.join(outputPath + "files/")
    pressureContourSwarm.save( fullpath + "pressureContourCoords" + str(step).zfill(5))
    pressureOnContour.save( fullpath + "pressureContourVals" + str(step).zfill(5))
    
    #these should / may go out of scope once the function returns anyway
    del mask
    del pointsRetain
    del pressureOnContour
    del pressureContourSwarm
    

In [121]:
#create velocity magnitude fn
velMagFn = uw.function.math.sqrt( uw.function.math.dot( velocityField, velocityField ) )

In [122]:
def files_update():
    
    #get all coords for the marker
    marker.swarm.shadow_particles_fetch()
    dims = marker.swarm.particleCoordinates.data.shape[1]
    pc = np.append(marker.swarm.particleCoordinates.data,
                       marker.swarm.particleCoordinates.data_shadow)
    all_coords = pc.reshape(-1,dims)
    pd = np.append(marker.director.data,
                       marker.director.data_shadow)
    all_directs = pd.reshape(-1,dims)
    
    #We don't evaluate on the fault but at a small distance in the normal direction
    ds = ndp.faultThickness/2.
    markerEvalPoints = all_coords - all_directs[...]*ds
    ds2 = 1.5*ndp.faultThickness #distace into the mantle
    mantleEvalPoints = all_coords - all_directs[...]*ds2
    
    
    #Create the temp Swarm
    markerEval = uw.swarm.Swarm( mesh=mesh )
    markerEval.add_particles_with_coordinates(markerEvalPoints)
    
    mantleEval= uw.swarm.Swarm( mesh=mesh )
    mantleEval.add_particles_with_coordinates(mantleEvalPoints)
    
    #Create the temp Swarm vars.
    
    markerVisc = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerSr2Inv = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerSrxx = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerSryy = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerEvalPoints = uw.swarm.SwarmVariable(markerEval, 'double', 2)
    markerYieldStress = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerTemp = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerPressure = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerVelMag = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    markerAge = uw.swarm.SwarmVariable(markerEval, 'double', 1)
    
    mantleVelY = uw.swarm.SwarmVariable(mantleEval, 'double', 1)
    mantleTemp = uw.swarm.SwarmVariable(mantleEval, 'double', 1)
    mantleEvalPoints = uw.swarm.SwarmVariable(mantleEval, 'double', 2)

    uplithEval = surfaceLine.swarm.particleCoordinates.data  - [0.0, 15e3/sf.lengthScale]
    midlithEval = surfaceLine.swarm.particleCoordinates.data  - [0.0, 30e3/sf.lengthScale]
    
    
    #define any NN interps we'll need

    ix1, weights1, d1 = nn_evaluation(swarm.particleCoordinates.data, markerEval.particleCoordinates.data, n=5, weighted=True)
    ix2, weights2, d2 = nn_evaluation(swarm.particleCoordinates.data, surfaceLine.swarm.particleCoordinates.data, n=5, weighted=True)


    #safeguarding - prob not necessary, partial relic of earlier implementation 
    #Fault stuff  
    if markerEval.particleCoordinates.data.shape[0]:
        markerEvalPoints.data[:] = markerEval.particleCoordinates.data

        #Update the data:
        if len(weights1): #i.e. if there is anu markerLine in this proc
            markerVisc.data[:,0] =  np.average(viscosityMapFn.evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)
        markerSr2Inv.data[:] = strainRate_2ndInvariant.evaluate(markerEval)
        markerSrxx.data[:] = symStrainrate[0].evaluate(markerEval)
        markerSryy.data[:] = symStrainrate[1].evaluate(markerEval)
        markerYieldStress.data[:] = interfaceysf.evaluate(markerEval)
        markerPressure.data[:] = pressureField.evaluate(markerEval) #only relavent for thermal models
        if md.thermal:
            markerTemp.data[:] = temperatureFn.evaluate(markerEval) #only relavent for thermal models
        markerVelMag.data[:] = velMagFn.evaluate(markerEval)
        markerAge.data[:] = lithAgeField.evaluate(markerEval)
        
    #Mantle stuff
    if mantleEval.particleCoordinates.data.shape[0]:
        mantleEvalPoints.data[:] = mantleEval.particleCoordinates.data
        mantleVelY.data[:] = velocityField[1].evaluate(mantleEval)
        mantleTemp.data[:] = temperatureField.evaluate(mantleEval)
    
    #Surface stuff 
    
    surfVelx.data[:] = velocityField[0].evaluate(surfaceLine.swarm)
    surfStrain.data[:] = symStrainrate[1].evaluate(surfaceLine.swarm)
    surfPres.data[:] = pressureField.evaluate(surfaceLine.swarm)
    
    if md.thermal:
        surfTgrad.data[:] = temperatureFn.fn_gradient[1].evaluate(surfaceLine.swarm)

    if len(weights2):
        surfVisc.data[:,0] =    np.average(viscosityMapFn.evaluate(swarm)[:,0][ix2], weights=weights2, axis=len((weights2.shape)) - 1)
        if md.thermal:
            surfTgrad.data[:] = temperatureFn.fn_gradient[1].evaluate(surfaceLine.swarm)

    
    #Mid lith stuff
    
    if midlithEval.shape[0]:
        midlithEvalEvalPoints.data[:] = midlithEval
        midlithVisc.data[:] = mantleRheologyFn.evaluate(midlithEval) #use mantleRheology to conserve resources
        midlithSr2Inv.data[:] = strainRate_2ndInvariant.evaluate(midlithEval)
        midlithStrainTens.data[:] = symStrainrate.evaluate(midlithEval)
        midlithVelX.data[:] = velocityField[0].evaluate(midlithEval)
        
    if uplithEval.shape[0]:
        uplithEvalEvalPoints.data[:] = uplithEval
        uplithVisc.data[:] = mantleRheologyFn.evaluate(uplithEval) #use mantleRheology to conserve resources
        uplithSr2Inv.data[:] = strainRate_2ndInvariant.evaluate(uplithEval)
        uplithStrainTens.data[:] = symStrainrate.evaluate(uplithEval)
        uplithVelX.data[:] = velocityField[0].evaluate(uplithEval)
        
    
    
    #Save the files:
    fullpath = os.path.join(outputPath + "files/")
    
    #marker
    markerEvalPoints.save( fullpath + "markerEval" + str(step).zfill(5))
    markerYieldStress.save( fullpath + "markerYieldStress" + str(step).zfill(5))
    markerTemp.save( fullpath + "markerTemp" + str(step).zfill(5))
    markerVisc.save( fullpath + "markerVisc" + str(step).zfill(5))
    markerSr2Inv.save( fullpath + "markerSr2Inv" + str(step).zfill(5))
    markerSrxx.save( fullpath + "markerSrxx" + str(step).zfill(5))
    markerSryy.save( fullpath + "markerSryy" + str(step).zfill(5))
    markerPressure.save( fullpath + "markerPressure" + str(step).zfill(5))
    markerVelMag.save( fullpath + "markerVelMag" + str(step).zfill(5))
    markerAge.save( fullpath + "markerAge" + str(step).zfill(5))
    
    #mantle
    mantleVelY.save( fullpath + "mantleVelY" + str(step).zfill(5))
    mantleTemp.save( fullpath + "mantleTemp" + str(step).zfill(5))
    mantleEvalPoints.save( fullpath + "mantleEval" + str(step).zfill(5))
    
    #suface
    surfVelx.save( fullpath + "surfVelx" + str(step).zfill(5))
    surfStrain.save( fullpath + "surfStrain" + str(step).zfill(5))
    surfVisc.save( fullpath + "surfVisc" + str(step).zfill(5))
    surfTgrad.save( fullpath + "surfTgrad" + str(step).zfill(5))
    surfPres.save( fullpath + "surfPres" + str(step).zfill(5))
    
    #mid lith
    midlithEvalEvalPoints.save( fullpath + "midlithEvalEvalPoints" + str(step).zfill(5))
    midlithVisc.save( fullpath + "midlithVisc" + str(step).zfill(5))
    midlithSr2Inv.save( fullpath + "midlithSr2Inv" + str(step).zfill(5))
    midlithStrainTens.save( fullpath + "midlithStrainTens" + str(step).zfill(5))
    midlithVelX.save(fullpath + "midlithVelx" + str(step).zfill(5))
    
    #up lith
    uplithEvalEvalPoints.save( fullpath + "uplithEvalEvalPoints" + str(step).zfill(5))
    uplithVisc.save( fullpath + "uplithVisc" + str(step).zfill(5))
    uplithSr2Inv.save( fullpath + "uplithSr2Inv" + str(step).zfill(5))
    uplithStrainTens.save( fullpath + "uplithStrainTens" + str(step).zfill(5))
    uplithVelX.save(fullpath + "uplithVelx" + str(step).zfill(5))

    
    uw.barrier()
    #big delete - all components of the temporary swarm 
    
    del pc
    del all_coords
    del pd
    del all_directs
    del markerEvalPoints
    del markerVisc
    del markerSr2Inv
    del markerSrxx 
    del markerSryy
    del markerYieldStress
    del markerEval
    del mantleEval
    del mantleVelY
    del mantleEvalPoints

In [123]:
#proximityVariable.evaluate(swarm)

In [124]:
def swarm_to_mesh_update():
    #define any NN interps we'll need
    ix1, weights1, d1 = nn_evaluation(swarm.particleCoordinates.data, mesh.data, n=5, weighted=True)
    
    
    #rebuild any mesh vars that are not self-updating
    proximityField.data[:,0] =  np.average(proximityVariable.evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)
    viscosityField.data[:,0] =  np.average(viscosityMapFn.evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)
    strainRateField.data[:] = strainRate_2ndInvariant.evaluate(mesh)
    
    stressFieldX.data[:,0] = np.average(stressXVariableFn.evaluate(swarm)[ix1][:,:,0],weights=weights1, axis=1)
    stressFieldMag.data[:,0] = np.average(stressMagVariableFn.evaluate(swarm)[ix1][:,:,0],weights=weights1, axis=1)
    
    lithAgeField.data[:,0] =  np.average(ageVariable.evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)

    
    #ReBuild the principal stress vector
    ssr = sym_strainRate.evaluate(mesh) #this does need to be here, 
        
    principalAngles  = np.apply_along_axis(eig2d, 1, ssr[:, :])[:,2]
    eig1.data[:,0] = np.cos(np.radians(principalAngles - 90.)) #most compressive 
    eig1.data[:,1] = np.sin(np.radians(principalAngles - 90.))
    eig2.data[:,0] = np.cos(np.radians(principalAngles ))      #most extensive
    eig2.data[:,1] = np.sin(np.radians(principalAngles ))


In [125]:
def xdmfs_update():
        
    fullpath = os.path.join(outputPath + "xdmf/")
    #if not os.path.exists(fullpath+"mesh.h5"):
    #    _mH = mesh.save(fullpath+"mesh.h5")
    
    try:
        _mH
    except:
        _mH = mesh.save(fullpath+"mesh.h5")
    
    
    #Part 1
    mh = _mH
    vH = velocityField.save(fullpath + "velocity_" + str(step) +".h5")
    if md.thermal:
        tH = temperatureFn.save(fullpath + "temp_" + str(step) + ".h5")
    srH = strainRateField.save(fullpath + "strainrate_" + str(step) +".h5")
    viscH = viscosityField.save(fullpath + "visc_" + str(step) + ".h5")
    
    eH = eig1.save(fullpath + "eig_" + str(step) + ".h5")
    eH2 = eig2.save(fullpath + "eig2_" + str(step) + ".h5")
    sigXX = stressFieldX.save(fullpath + "sigXX_" + str(step) + ".h5")
    sigII = stressFieldMag.save(fullpath + "sigII_" + str(step) + ".h5")
    presH = pressureField.save(fullpath + "pressure_" + str(step) +".h5")
   
    #part2
    
    velocityField.xdmf(fullpath + "velocity_" + str(step), vH, 'velocity', mh, 'mesh', modeltime=time)
    if md.thermal:
        temperatureFn.xdmf(fullpath + "temp_" + str(step), tH, 'temperature', mh, 'mesh', modeltime=time)
    strainRateField.xdmf(fullpath + "strainrate_" + str(step), srH, 'strainrate', mh, 'mesh', modeltime=time)
    viscosityField.xdmf(fullpath + "visc_" + str(step), viscH, 'visc', mh, 'mesh', modeltime=time)
    
    eig1.xdmf(fullpath + "eig_" + str(step), eH, 'eig', mh, 'mesh', modeltime=time)
    eig2.xdmf(fullpath + "eig2_" + str(step), eH2, 'eig2', mh, 'mesh', modeltime=time)
    stressFieldX.xdmf(fullpath + "sigXX_" + str(step), sigXX, 'sigXX', mh, 'mesh', modeltime=time)
    stressFieldMag.xdmf(fullpath + "sigII_" + str(step), sigII, 'sigII', mh, 'mesh', modeltime=time)
    pressureField.xdmf(fullpath + "pressure_" + str(step), presH, 'pressure', mh, 'mesh', modeltime=time)

    
    
    #temp 
    
    #restrictionFnField.data[:] = szRfn150.evaluate(mesh)
    #restrict = restrictionFnField.save(fullpath + "restrict_" + str(step) + ".h5")
    #restrictionFnField.xdmf(fullpath + "restrict_" + str(step), restrict, 'restrict', mh, 'mesh', modeltime=time)



In [126]:
def update_wedge_pos():
    pass
    

In [133]:
def valuesUpdateFn():
    
    """ 
    Assumes global variables:
    * time
    * step 
    ...
    + many functions
    """
    
    
    #save the time and step
    valuesDict.timeAtSave.append(time) 
    valuesDict.stepAtSave.append(step)
    
    #Save plate boundary locs.
    #changes here (fn.value) should propagate through any connected funtions
    _szVelGrads.reset()
    dummyFn = _szVelGrads.evaluate(tWalls)
    dummyVal = _szVelGrads.min_global_auxiliary()[0][0]
    prevVal = subZoneLoc.value
    subZoneLoc.value = (prevVal + dummyVal)/2. #a lazy smoothing of the update to help dampen fluctuations
    
    _spRidgeVelGrads.reset()
    dummyFn = _spRidgeVelGrads.evaluate(tWalls)
    dummyVal = _spRidgeVelGrads.max_global_auxiliary()[0][0]
    spRidgeLoc.value = dummyVal
    
    _opRidgeVelGrads.reset()
    dummyFn = _opRidgeVelGrads.evaluate(tWalls)
    dummyVal = _opRidgeVelGrads.max_global_auxiliary()[0][0]
    opRidgeLoc.value = dummyVal

    valuesDict.opRidgeLoc.append(round(opRidgeLoc.value, 5))
    valuesDict.subZoneLoc.append(round(subZoneLoc.value, 5))
    valuesDict.spRidgeLoc.append(round(spRidgeLoc.value, 5))

    
    #Evaluate /save maxMins. These need resetting and evaluation
    _mmVxSurf.reset()
    dummyFn = _mmVxSurf.evaluate(mesh)
        
    _mmVxSurf.reset()
    dummyFn = _mmVxSurf.evaluate(mesh)
    
    _wedgeHotCorner.reset()
    dummyFn = _wedgeHotCorner.evaluate(mesh)
    
    
    valuesDict.maxVxSurf.append(round(_mmVxSurf.max_global(), 5))
    valuesDict.minVxSurf.append(round(_mmVxSurf.min_global(), 5))
    
    valuesDict.wedgeHotCornerX.append(round(_wedgeHotCorner.min_global_auxiliary()[0][0], 5))
    valuesDict.wedgeHotCornerY.append(round(_wedgeHotCorner.min_global_auxiliary()[0][1], 5))



    #Evaluate /save integrals 
    valuesDict.areaintLith.append(round(_areaintLith.evaluate()[0] , 5))
    valuesDict.tempInt.append(round(_tempint.evaluate()[0] , 5))
    valuesDict.vdintLith.append(round( _vdintLith.evaluate()[0], 5))    
    valuesDict.potentialWork.append(round(_dwint.evaluate()[0], 5))
    valuesDict.vdintLower.append(round(_vdintLower.evaluate()[0], 5))
    valuesDict.areaintLower.append(round(_areaintLower.evaluate()[0], 5)) 
    valuesDict.vdintLowerBending.append(round(_vdintLowerBending.evaluate()[0], 5))
    valuesDict.areaintLowerBending.append(round(_areaintLowerBending.evaluate()[0], 5)) 
    valuesDict.vdintUM.append(round(_vdintUM.evaluate()[0], 5))
    valuesDict.vdintLM.append(round(_vdintLM.evaluate()[0], 5))
    valuesDict.areaintSlabLM.append(round(_areaintSlabLM.evaluate()[0], 5))
    valuesDict.vdintSlabLM.append(round(_vdintSlabLM.evaluate()[0], 5))
    valuesDict.areaintInterface.append(round(_areaintInterface.evaluate()[0], 5))
    valuesDict.vdintInterface.append(round(_vdintInterface.evaluate()[0], 5))
    valuesDict.nusseltTop.append(round(_nuTop.evaluate()[0], 5))
    valuesDict.critWedgeTempArea.append(round(_areaintcritWedgeTemp.evaluate()[0], 5))
    

    
    #save
    if uw.rank()==0:
        fullpath = os.path.join(outputPath + "model_data")
        #the '**' is important
        np.savez(fullpath, **valuesDict)

## Run any of the required update functions

In [134]:
swarm_to_mesh_update()

## Main loop

In [135]:
time = cp.time()  # Initial time
step = cp.step()   # Initial timestep
files_freq  = md.filesMy*(3600.*365.*24.)/sf.time  #applies to files and gldbs
files_this_step = False
next_image_step = (np.floor(time/files_freq)+ 1.) *files_freq 


In [ ]:
#checkpoint at time zero
if not cp.restart:
    cp.saveObjs(step, time)
    cp.saveDicts(step, time)

In [ ]:
while step < md.maxSteps:
    
    solver.solve(nonLinearIterate=True, nonLinearTolerance=md.nltol, callback_post_solve = pressure_calibrate)

    
    # main
    time, step, dt, files_this_step, next_image_step = main_update(next_image_step)

    # age
    age_update()
    
   
    #particles
    #update static mesh variables
    #markers / markerLines
    if step % md.swarmUpdate == 0:
        swarm_update()
        swarm_to_mesh_update()
        uw.barrier()
        markerLine_update()
        fault_update()
    
    #Heal the markers
    if step % 5 == 0:
        heal_markers_update(5, 4)
    
    #checkpoint
    if step % md.checkpointEvery == 0:
        cp.saveObjs(step, time)
        cp.saveDicts(step, time)
        
        
    #files
    if files_this_step:
        files_update()
        pressure_contour_points_update()

 
    #xdmfs
    if files_this_step:
        xdmfs_update()

               
    #output values
    if files_this_step:
        valuesUpdateFn()
        
            
    #Viz
    if files_this_step:
        viz_update() 
  
        
    print 'step =',step
    

    

print 'step =',step

step = 1


In [ ]:
#ix1, weights1, d1 = nn_evaluation(swarm.particleCoordinates.data, mesh.data[tWalls.data], n=5, weighted=True)
#surfStressData =    np.average(stressFn[1].evaluate(swarm)[:,0][ix1], weights=weights1, axis=len((weights1.shape)) - 1)

In [ ]:
#vx = velocityField[0].evaluate(mesh.data[tWalls.data])
#press = pressureField.evaluate(mesh.data[tWalls.data])

In [ ]:
#%pylab inline

# new style method 1; unpack the axes
#fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=False)
#ax1.plot((-1.*surfStressData*sf.stress), c='r')
#ax1.plot((press*sf.stress))
#ax2.plot(vx)


In [ ]:
#We don't evaluate on the marker but, here
#ds = ndp.faultThickness/2.
#markerEval = marker.swarm.particleCoordinates.data - marker.director.data[...]*ds

#marker2 = markerLine2D(mesh, velocityField,markerEval[:,0], markerEval[:,1], ndp.faultThickness,  6)

#with marker2.swarm.deform_swarm():
#    marker2.swarm.particleCoordinates.data[:] += marker2.director.data*ndp.faultThickness

In [ ]:
dims = marker.swarm.particleCoordinates.data.shape[1]
pc = np.append(marker.swarm.particleCoordinates.data,
                   marker.swarm.particleCoordinates.data_shadow)
all_coords = pc.reshape(-1,dims)
pd = np.append(marker.director.data,
                   marker.director.data_shadow)
all_directs = pd.reshape(-1,dims)

#We don't evaluate on the fault but at a small distance in the normal direction
ds = ndp.faultThickness/2.
markerEvalPoints = all_coords - all_directs[...]*ds
ds2 = 3.*ndp.faultThickness
mantleEvalPoints = all_coords - all_directs[...]*ds2


#Create the temp Swarm
markerEval = uw.swarm.Swarm( mesh=mesh )
markerEval.add_particles_with_coordinates(markerEvalPoints)

mantleEval= uw.swarm.Swarm( mesh=mesh )
mantleEval.add_particles_with_coordinates(mantleEvalPoints)

In [119]:
#fig1 = glucifer.Figure(figsize=(300*np.round(md.aspectRatio,2),300))

#fig2.append( glucifer.objects.Points(swarm, materialVariable, pointSize=2, fn_mask=vizVariable))
#fig1.append( glucifer.objects.Points(swarm, proximityVariable,pointSize=2))
#fig1.append( glucifer.objects.Points(markerEval))
#fig1.append( glucifer.objects.Points(swarm, proxyTempVariable))

#fig1.append( glucifer.objects.Points(fault.swarm))

#fig2.append( glucifer.objects.Points(testMarker.swarm))

#fig1.show()
#fig1.append( glucifer.objects.VectorArrows(mesh, velocityField, arrowHead=5, scaling=0.00005))
#fig1.save_database('test.gldb')

In [ ]:
#import pickle
##with open('dp.pkl', 'rb') as fp:
#    dill = pickle.load(fp)
#dill

In [5]:
#(160./96)*750.

1250.0

In [ ]:
subZoneLocFn = fn.misc.constant(subZoneLoc)
szRfn150 = 1. - cosine_taper(fn.math.abs(xFn - subZoneLocFn),pbbound, pbwidth)


_testMM = fn.view.min_max(szRfn150*xFn**2, fn_auxiliary=xFn)
dummyFn = _testMM.evaluate(tWalls)

print(_testMM.max_global())

_tempint = volumeint(szRfn150*xFn)
print(_tempint.evaluate()[0])

#reset underlying value
subZoneLocFn.value = 0.7

#shallow reevaluate
_testMM.reset()

dummyFn = _testMM.evaluate(tWalls)
print(_testMM.max_global(), _testMM.max_global_auxiliary())

print(_tempint.evaluate()[0])

#full reevaluate
_testMM = fn.view.min_max(szRfn150*xFn**2, fn_auxiliary=xFn)
dummyFn = _testMM.evaluate(tWalls)
print(_testMM.max_global(), _testMM.max_global_auxiliary())
_tempint = volumeint(szRfn150*xFn)
print(_tempint.evaluate()[0])